# Retrieval Augmented Generation using Qdrant HuggingFace embeddings and Langchain and Evaluate the Response Generated using OpenAI

In [1]:
! pip  install -qU openai langchain PyPDF2 transformers tiktoken pinecone-client sentence-transformers qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
! pip install -U pytesseract
! pip install pdf2image

In [3]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.6 MB/s eta 0:00:00


In [4]:
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.qdrant import Qdrant
import PyPDF2
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from uuid import uuid4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from time import time,sleep
import openai
from langchain import VectorDBQA
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
#
import os
import json
#
import io
import pytesseract
import shutil
import random
from PIL import Image
from pdf2image import convert_from_path
from configparser import ConfigParser
#
from typing import Dict, List, Optional, Tuple
from langchain.docstore.document import Document
from langchain.vectorstores import Qdrant
import pandas as pd
import numpy as np
#

In [5]:
import openai
from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] =getpass()

··········


In [21]:
#response
([{'query': 'what is a neccessary Part of the Capital Structure according to Fred Wilson?',
   'result': 'According to Fred Wilson, a necessary part of the capital structure is employee equity or ESOPs (Employee Stock Ownership Plans). He states, "I can\'t think of a term sheet that we have issued that didn\'t have a specific provision for employee equity." This suggests that offering employee equity is a prerequisite for venture capitalists and is commonly included in deals with startups.',
   'source_documents': [Document(page_content="A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures", metadata={'chunk': 0, 'text': "A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures", 'page number': 'Page10'}),
    Document(page_content='The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time', metadata={'chunk': 0, 'text': 'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time', 'page number': 'Page15'}),
    Document(page_content='Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity', metadata={'chunk': 0, 'text': 'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity', 'page number': 'Page42'}),
    Document(page_content='Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting', metadata={'chunk': 0, 'text': 'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting', 'page number': 'Page43'}),
    Document(page_content='Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage', metadata={'chunk': 1, 'text': 'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage', 'page number': 'Page11'})]},
  {'query': 'When to Communicate Options to Employees: % versus $',
   'result': 'The context does not provide information about when to communicate options to employees as a percentage versus a dollar value.',
   'source_documents': [Document(page_content='Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather', metadata={'chunk': 0, 'text': 'Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather', 'page number': 'Page12'}),
    Document(page_content='value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', metadata={'chunk': 1, 'text': 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', 'page number': 'Page12'}),
    Document(page_content='Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success', metadata={'chunk': 0, 'text': 'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success', 'page number': 'Page8'}),
    Document(page_content='0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS', metadata={'chunk': 0, 'text': '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS', 'page number': 'Page24'}),
    Document(page_content='3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule', metadata={'chunk': 0, 'text': '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule', 'page number': 'Page21'})]},
  {'query': 'When to Create an ESOP?',
   'result': 'ESOPs should be created during the seed stage or early-VC stage of a startup.',
   'source_documents': [Document(page_content='When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted', metadata={'chunk': 0, 'text': 'When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted', 'page number': 'Page11'}),
    Document(page_content='Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired', metadata={'chunk': 0, 'text': 'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired', 'page number': 'Page14'}),
    Document(page_content='members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP', metadata={'chunk': 1, 'text': 'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP', 'page number': 'Page16'}),
    Document(page_content='Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language', metadata={'chunk': 0, 'text': 'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language', 'page number': 'Page34'}),
    Document(page_content='ESOPS FOR THE LONG TERM  Part IV', metadata={'chunk': 0, 'text': 'ESOPS FOR THE LONG TERM  Part IV', 'page number': 'Page35'})]},
  {'query': 'What are the Common Terms in an Options Package?',
   'result': 'The common terms in an options package are:\n1. Number of Shares: The total number of options granted to an employee, and the maximum number of shares that employee has access to.\n2. Strike Price: The price the employee must pay to purchase each share if and when the employee chooses to exercise the option.\n3. Vesting Schedule: The timeline over which the options become wholly owned and exercisable by the employee, no longer subject to repurchase by the company.\n4. Cliff Period: The trial period during which no vesting occurs; in this period vesting accrues, but the total effect of this vesting is realized immediately after the cliff.\n5. Expiration Date: The last date on which the options may be exercised and converted into common shares by the employee.\n\nQuotes supporting the answer:\n- "Number of Shares: The total number of options granted to an employee, and therefore the maximum number of shares that employee has access to."\n- "Strike Price: The price the employee must pay to purchase each share if and when the employee chooses to exercise the option."\n- "Vesting Schedule: The timeline over which the options become wholly owned and exercisable by the employee (no longer subject to repurchase by the company)."\n- "Cliff Period: The trial period during which no vesting occurs; in this period vesting accrues, but the total effect of this vesting is realized immediately after the cliff."\n- "Expiration Date: The last date on which the options may be exercised and converted into common shares by the employee."',
   'source_documents': [Document(page_content='Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants', metadata={'chunk': 0, 'text': 'Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants', 'page number': 'Page7'}),
    Document(page_content='Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time', metadata={'chunk': 1, 'text': 'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time', 'page number': 'Page5'}),
Document(page_content='What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \
    'chunk': [
        'Page',
        'Page7',
        'Page8',
    ]
}
   'Page9',
   'Page10',
   'Page11',
   'Page12',
   'Page13',
   'Page14',
   'Page15',
   'Page16',
   'Page17',
   'Page18',
   'Page19',
   'Page20',
   'Page21',
   'Page22',
   'Page23',
   'Page24',
   'Page25',
   'Page26',
   'Page27',
   'Page28',
   'Page29',
   'Page30',
   'Page31',
   'Page32',
   'Page33',
   'Page34',
   'Page35',
   'Page36',
   'Page37',
   'Page38',
   'Page39',
   'Page40',
   'Page41',
['Page42',
 'Page43',
['Page44'],
['Startup Employee Stock \nOptions Plans (ESOPs)  \n \nOverview and Best Practices',
 'Table of Contents  \nPart I: Intro to Options Plans  \n•What is an ESOP?  \n•What is an Option?  \n•Lifecycle of a Startup ESOP  \n•Common Terms in an Options Package  \n•Why Issue Options to Employees ? \n–A Defining Characteristic of Startup Culture  \n–A Necessary Part of the'],
 ['•Important Takeaways  Part III : The Fine Print – Terms  \
   'What is an ESOP?  \n•An Employee Stock Options Plan (ESOP)  \n \n•An allocation  of shares that will be granted to employees in the future \nin the form of stock options  \n–How much equity should we set aside for employees?  \n \n•A plan for how these options will be distributed:  \n–How many shares will individual employees receive?  \n–What terms will govern these grants?  \n \n•The plan is as important as the allocation!',
   'What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity',
   'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time',
   'Lifecycle of a Startup ESOP  \n•Founders and early investors  create an ESOP by setting aside a \npercentage of shares to be granted to future employees  \n•Management and the Board of Directors  issue these shares to \nemployees as options packages granted for hiring, promotion and \nretention  \n•Employees  receive all of their options upfront, but the company \nmaintains a right to force forfeit that diminishes over time through a \nprocess called “vesting”  \n•Options are exercised  by employees when the company is acquired \nor taken public. The employee pays the “strike price” to acquire the \nshares, but those shares are now marketable at a higher value',
   'Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants',
   'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success',
   'The defining difference between Silicon Valley companies and almost \nevery other industry in the U.S. is the virtually universal practice among \ntech companies of distributing meaningful equity (usually in the form of \nstock options) to ordinary employees .  A Defining Characteristic of Startup Culture  \nSteven Johnson, Technology Writer  \n•Startups are a unique case . Unlike at larger corporations, employee \nownership is an essential element of startup communities and culture  \n–As high -risk/high -reward enterprises, startups use options to align \nemployee compensation with the risk -prone mentality of the \nbusiness  \n–Startups seeking to achieve a “big exit” use options to align all \nemployees to drive toward this desired outcome',
   "A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures",
   'When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted',
   'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage',
   'Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather',
   'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .',
   'HOW MUCH TO GRANT  Part II',
   'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired',
   'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time',
   '1. How Much Equity to Set Aside in the ESOP?  \n•Founders  are top managers \nwith functional roles that are \nnot redundant  \n•DNA  of the company is in a \nnon-technical industry; no \ntechnology emphasis  \n•Geographic focus  is on a \nregion that is not considered a \nstartup hub  \n•Hiring needs are primarily \nadministrative or back -office \nteam members  <15%  20% 25% Generally non-founder employees  get 15 -20% of the company, with some \ncompanies issuing up to 25% (and a current trend toward bigger ESOPs)  \n•Founders  are top mangers, \nbut hire others to fill key \ntechnical or functional roles  \n•DNA  of the company is in \napplying technology to a non -\ntechnical industry  \n•Geographic focus  is either \nwholly or partially in a startup \nhub \n•Hiring needs are primarily \nsales and marketing team \nmembers  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a',
   'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP',
   '2. A Typical Distribution Schedule  \nSeniority  Equity Allocation  \nFirst 10 Employees  10% \nNext 20 Employees  5% \nNext 50 Employees  5% \n•Early -stage equity grants are always a negotiation, but generally:  \n–CTO: 1 -5% \n–Key Developer or Engineer: 1 -2% \n–Other Functional Team Member: 0.5 -1.5%  \n–No non -founding member of the senior team should receive over 10%',
   'The Bottom -Up Process  \n1.Segment your human resources  \n2.Create pay multipliers for each job function  \n3.Determine the dollar value of an options grant  \n4.Determine the current share price  \n5.Calculate options grants',
   '1. Segment Your Human Resources  \nSegment  Roles  Award  \nin % or $?  \nFounders  Co-Founders  % \nFounding Team  Technical & Product  % \nEarly -Stage Hires  Senior Team  (C-Level)  $ \nDirectors / VPs  $ \nFunctional / Technical Team  $ \nSupport Team  $ \n•Think about the early -stage team in three segments  \n–Founders : the founding partners in the business  \n–Founding Team : the people you hire to build your product  \n–Early -Stage Hires : the team you build as you raise capital  \n•As early as possible, begin communicating options grants in terms of \ndollars rather than percentages',
   '2. Establish Pay Multipliers for Each Role  \nSegment  Roles  Award  \nin % or $?  Options Multiplier  \n(Multiple  of Salary)  \nFounders  Co-Founders  % -- \nFounding Team  Technical & Product  % Negotiated  \nEarly -Stage Hires  Senior Team  (C-Level)  $ 0.5x – 1.0x base  salary  \nDirectors / VPs  $ 0.25x – 0.5x base salary  \nFunctional / Technical Team  $ 0.1x – 0.25x base  salary  \nSupport Team  $ 0.0x – 0.1x base salary  \n•After segmenting the early -stage team, establish a standardized \nmultiple of base salary to be granted as an option package  \n–These ranges provide a ballpark grant for each role  \n–Get away from actively negotiating packages with employees  \n–Be transparent about this methodology  \nSource: This range of multiples was provided by Fred Wilson of Union Square Ventures, \nwhich he created in collaboration with a leading HR consulting firm.',
   '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule',
   '4. Determine the Current Share Price  \nLatest “True” Valuation  \nFully -Diluted Share Count  Current Share Price  = What is the company actually worth today?  Or, at what \nvaluation could you raise capital or sell the company? This \nis an internal valuation (not published). Things to consider:  \n-The latest financing round  \n-Any bonafide  acquisition offer  \n-Comparable company valuations / DCF analysis  \nHow many shares are outstanding?  This should include \nall shares issued, as well as any dilutive securities:  \n-Common shares  \n-Preferred shares  \n-Shares authorized for issuance  \n-Options or warrants',
   '5. Calculate the Options Grant  \nOptions Grant  \n(Number of Shares)  = Dollar Value  \nof Options Grant  \nCurrent Share Price',
   '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS',
   'Important Takeaways  \n•Top-down planning (“the allocation”)  \n–Holistically consider what percentage of the company should \nbelong to employees  \n–Allocate these shares to an ESOP  \n•Bottom -up planning (“the plan”)  \n–Specifically consider how much equity each employee should be \nawarded  \n–Use this framework for individual equity grants  \n•Key Takeaway : Get away from ad hoc equity awards and personal \nnegotiations by standardizing both the amount of equity available to \nemployees and the process by which packages are awarded',
   'THE FINE PRINT – TERMS  Part III',
   'Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”',
   'Vesting Schedule  \n•Vesting protects the company  \n–It stages the accrual of options, mitigating the risk that an employee \nwill depart with an undeserved equity stake  \n–Continually incentivizes employees as they earn their options \npackage over the course of the vesting period  \n•Standard vesting period: 4 years  \n–Alternatively, some companies offer 3 -year vesting, although with a \nshorter vesting period they may offer a smaller options package  The vesting schedule is the timetable over which an employee accrues the \nright to keep the options that have been awarded',
   'The Cliff  \n•The cliff protects the company  \n•This trial period exists so that the company can avoid giving shares to \nemployees immediately, reserving the opportunity part ways if the \nemployee does not perform well  \n•Standard cliff: 1 year  \n–Alternatively, some companies require a 2 -year cliff. However, in \nstartup communities where employees know what is “standard” this \ncan cause significant discontent  \n–Firing an employee right as their cliff is about to expire is legal, but \nunethical – begin to communicate discontent as early as possible \nduring the trail period  The trial period during which no vesting occurs; in this period vesting \naccrues, but the accrued shares are earned immediately after the cliff',
   '200 shares400 shares600 shares800 shares1,000 shares\n0 months 12 months 24 months 36 months 48 monthsExample: Standard Vesting with a Cliff  \n•New hire  \n•Granted options for 1,000 shares  \n•4-year vesting schedule with 1 -year cliff  \n–At 12 months earns 250 shares  \n–Then earns ~21 shares per month  \nMonths on the Job  Vested Options  \nThe cliff  Monthly vesting',
   'Vesting in a Liquidity Event  \nThe terms of an options package should specifically address what \nhappens in a liquidity event such as an IPO or acquisition  \n•Vested options generally become exercisable  \n•Unvested options can either:  \n–Roll over and remain unvested  \n–Receive accelerated vesting  \n•These options vest automatically upon change of control – not \ncommon for everyone, but typical for top employees  \n–Receive double -trigger vesting  \n•These options vest automatically upon change of control, but \nonly if the employee is terminated – not common for everyone, \nbut typical for employees who may be extraneous in a merger \n(CFO, General Counsel)',
   'Exercising Options  \n•Options are “in the money” if the share value is above the strike price  \n•To exercise an option, the employee pays the strike price to the \ncompany and receives shares for each option exercised  \n•If there has been a liquidity event  \n–In the case of an IPO , the employee can now sell these shares in \nthe public markets  \n–In the case of a cash acquisition , an employee’s shares are \nbought out at the cash offer price  \n–In the case of a stock acquisition , an employee’s shares are \nconverted to shares in the new company  Options are usually exercised following a liquidity event, but employees \ncan exercise them at any time after they vest',
   'Tax Considerations  \nOptions can have material tax consequences for employees; startups \nneed to understand the tax implications and be upfront with new hires  \n•Tax treatment of options will vary by region  \n•Tax treatment will vary by the type option security issued  \n•Several possibilities:  \n–Taxable event upon grant  \n–Taxable event upon leaving company  \n–Taxable event upon exercise  \n–No effective tax event  \n•It is important for employees to fully understand the tax implications of \nthe securities they are receiving',
   'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language',
   'ESOPS FOR THE LONG TERM  Part IV',
   'Retention Grants  \n•Retention grants are used to incentive employees to stay with the \ncompany beyond their 4 -year vesting period: a “top off”  \n–Do not wait until an employee is fully vested to make a retention \ngrant! This creates a natural exit incentive at the end of 4 years  \n–Instead, consider making smaller retention grants every 2 years  \n•A framework for retention grants:  \nGrant Date  Every 2  years, beginning 2 years after initial package  \nOptions Multiplier:  Use ½ of the employee’s standard options multiplier  \nSalary Baseline:  Use current salary  if employee has received a raise   \nValuation & Shares  Use current valuation and share count, if changed  \nRetention  \nGrant  = Current \nSalary  New (Latest) Valuation  \nDiluted Share Count  x 50% of Options \nMultiplier  ÷',
   'Discretionary Grants  \n•Bonuses Outstanding Performance  \n–These discretionary grants should only be made once per year, and \nonly offered to your top 10 -20% of employees whose \naccomplishments and performance are truly distinguished  \n•Promotions  \n–These discretionary grants should only be made to reward \npromotion. Bring the employee’s total equity up to the amount you \nwould pay to hire them today.',
   'Social Impact Considerations  \n•Companies focused on social impact goals and/or in developing \nmarkets may have unique ESOP considerations  \n•Consider two possible scenarios:  \n–Financial Inclusion Goals : Employee -friendly hiring practices and \nESOPs can help build local financial inclusion. If this is part of your \ncorporate mandate, consider the added social impact value of \noffering your options program to all levels of employees  \n–Local Ownership Culture : Conversely, in certain regions stock \noptions may have negligible value to employees, either because of \nrisk aversion, lack of liquidity, or lack of understanding. If this is the \ncase, it is not worth extensively offering options to employees who \nwould rather be paid in cash',
   'Options Modeling – Overview  \n•Creating an ESOP has lasting implications on startup capital structure  \n•While the guidelines provided in this presentation offer a roadmap to \nimplementing an options plan, it can also be helpful to build an options \nmodel to understand the long -term effects of this program  \n•Options models:  \n–Forecast hiring needs over time  \n–Estimate future valuation and fundraising needs  \n–Provide an estimate of employee ownership  \nNote: There is an accompanying excel file that provides a template for \nbuilding an options model similar to that described on the following slide',
   'Options Modeling – A Detailed Example  \nEmployee Stock Options Plan (ESOP) Model\nYear 0 Year 1 Year 2 Year 3 Year 4\nValue Shares Value Shares Shr. Price Value Shares Shr. Price Value Shares Shr. Price\nNA 10,000 $3M 10,000 $300 $5M 10,000 $500 $15M 12,000 $1,250\nOptions Total Equity Grant Total Equity Grant Total Equity Grant Total Equity\nSalary Multiplier Options (%) Amount Options (%) Amount Options (%) Amount Options (%)\nTotal Non-Founder Employee Ownership 6.50% 9.00% 11.64% 10.20%\nFounding Team\nCTO $120,000 1.00x 400 4.00% 400 4.00% $60,000 520 5.20% 520 4.33%\nProduct Manager $80,000 0.80x 250 2.50% 250 2.50% $32,000 314 3.14% 314 2.62%\nYear 1 Hires\nLead Engineer $80,000 0.50x $40,000 133 1.33% 133 1.33% $20,000 149 1.24%\nLead Designer $70,000 0.50x $35,000 117 1.17% 117 1.17% $17,500 131 1.09%\nYear 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires',
   'Year 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires\nMarketing Director $60,000 0.30x $18,000 14 0.12%\nEngineer $50,000 0.25x $12,500 10 0.08%\nSalesperson $40,000 0.10x $4,000 3 0.03%\nOffice Manager $30,000 0.10x $3,000 2 0.02%\nFounding team shares are \nnegotiated as a percentage of the \ncompany; all future employees \nreceive grants determined in dollars  \nRetention grants are awarded every two \nyears after an initial options package  The “true” valuation (an \ninternal estimate) increases \nmarginally as the company \nbuilds value…  …but increases significantly when \nthe company raises a new round, \nwhich also increases the share \ncount and dilutes all equity owners  \nFor all new hires after the \nfounding team, options grants \nare awarded based on the \nemployee’s options multiplier \nand baseline salary',
   'RESOURCES & FURTHER \nREADING  Part V',
   'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity',
   'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting',
   'This work  is licensed  under  the Creative  Commons  Attribution -\nShareAlike  4.0 International  License . To view a copy  of this license,  \nvisit http://creativecommons .org/licenses/by -sa/4.0/ or send  a letter  to \nCreative  Commons,  PO Box 1866 , Mountain  View,  CA 94042 , USA.'],
  [{'chunk': 0,
    'text': 'Startup Employee Stock \nOptions Plans (ESOPs)  \n \nOverview and Best Practices',
    'page number': 'Page1'},
   {'chunk': 0,
    'text': 'Table of Contents  \nPart I: Intro to Options Plans  \n•What is an ESOP?  \n•What is an Option?  \n•Lifecycle of a Startup ESOP  \n•Common Terms in an Options Package  \n•Why Issue Options to Employees ? \n–A Defining Characteristic of Startup Culture  \n–A Necessary Part of the Capital Structure  \n•When to Create an ESOP ? \n•Communicating Options to Employees: % versus $ \n \nPart II: How Much to Grant  \n•Two Approaches  \n•The Top -Down Process  \n–1. How Much Equity to Set Aside in the ESOP ? \n–2. A Typical Distribution Schedule  \n•The Bottom -Up Process  \n–1. Segment Your Human Resources  \n–2. Establish Pay Multipliers for Each Role \n–3. Determine the Dollar Value of the Options Grant  \n–4. Determine the Current Share Price  \n–5. Calculate the Options Grant  \n–An Example: Hiring a CTO  \n•Important Takeaways  Part III : The Fine Print – Terms  \n•Strike Price  \n•Vesting Schedule  \n•The Cliff  \n–Example: Standard Vesting w/ a Cliff \n•Vesting in a Liquidity Event  \n•Exercising Options',
    'page number': 'Page2'},
   {'chunk': 1,
    'text': '•Important Takeaways  Part III : The Fine Print – Terms  \n•Strike Price  \n•Vesting Schedule  \n•The Cliff  \n–Example: Standard Vesting w/ a Cliff \n•Vesting in a Liquidity Event  \n•Exercising Options  \n•Tax Considerations  \n•Legal Advice  \n \nPart IV: ESOPs for the Long Term  \n•Retention Grants  \n•Discretionary Grants  \n•Social Impact Considerations  \n•Options Modeling – Overview  \n•Options Modeling – A Detailed Example  \n \nPart V: Resources & Further Reading',
    'page number': 'Page2'},
   {'chunk': 0,
    'text': 'INTRO TO OPTIONS PLANS  Part I',
    'page number': 'Page3'},
   {'chunk': 0,
    'text': 'What is an ESOP?  \n•An Employee Stock Options Plan (ESOP)  \n \n•An allocation  of shares that will be granted to employees in the future \nin the form of stock options  \n–How much equity should we set aside for employees?  \n \n•A plan for how these options will be distributed:  \n–How many shares will individual employees receive?  \n–What terms will govern these grants?  \n \n•The plan is as important as the allocation!',
    'page number': 'Page4'},
   {'chunk': 0,
    'text': 'What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity',
    'page number': 'Page5'},
   {'chunk': 1,
    'text': 'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time',
    'page number': 'Page5'},
   {'chunk': 0,
    'text': 'Lifecycle of a Startup ESOP  \n•Founders and early investors  create an ESOP by setting aside a \npercentage of shares to be granted to future employees  \n•Management and the Board of Directors  issue these shares to \nemployees as options packages granted for hiring, promotion and \nretention  \n•Employees  receive all of their options upfront, but the company \nmaintains a right to force forfeit that diminishes over time through a \nprocess called “vesting”  \n•Options are exercised  by employees when the company is acquired \nor taken public. The employee pays the “strike price” to acquire the \nshares, but those shares are now marketable at a higher value',
    'page number': 'Page6'},
   {'chunk': 0,
    'text': 'Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants',
    'page number': 'Page7'},
   {'chunk': 0,
    'text': 'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success',
    'page number': 'Page8'},
   {'chunk': 0,
    'text': 'The defining difference between Silicon Valley companies and almost \nevery other industry in the U.S. is the virtually universal practice among \ntech companies of distributing meaningful equity (usually in the form of \nstock options) to ordinary employees .  A Defining Characteristic of Startup Culture  \nSteven Johnson, Technology Writer  \n•Startups are a unique case . Unlike at larger corporations, employee \nownership is an essential element of startup communities and culture  \n–As high -risk/high -reward enterprises, startups use options to align \nemployee compensation with the risk -prone mentality of the \nbusiness  \n–Startups seeking to achieve a “big exit” use options to align all \nemployees to drive toward this desired outcome',
    'page number': 'Page9'},
   {'chunk': 0,
    'text': "A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures",
    'page number': 'Page10'},
   {'chunk': 0,
    'text': 'When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted',
    'page number': 'Page11'},
   {'chunk': 1,
    'text': 'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage',
    'page number': 'Page11'},
   {'chunk': 0,
    'text': 'Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather',
    'page number': 'Page12'},
   {'chunk': 1,
    'text': 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .',
    'page number': 'Page12'},
   {'chunk': 0, 'text': 'HOW MUCH TO GRANT  Part II', 'page number': 'Page13'},
   {'chunk': 0,
    'text': 'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired',
    'page number': 'Page14'},
   {'chunk': 0,
    'text': 'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time',
    'page number': 'Page15'},
   {'chunk': 0,
    'text': '1. How Much Equity to Set Aside in the ESOP?  \n•Founders  are top managers \nwith functional roles that are \nnot redundant  \n•DNA  of the company is in a \nnon-technical industry; no \ntechnology emphasis  \n•Geographic focus  is on a \nregion that is not considered a \nstartup hub  \n•Hiring needs are primarily \nadministrative or back -office \nteam members  <15%  20% 25% Generally non-founder employees  get 15 -20% of the company, with some \ncompanies issuing up to 25% (and a current trend toward bigger ESOPs)  \n•Founders  are top mangers, \nbut hire others to fill key \ntechnical or functional roles  \n•DNA  of the company is in \napplying technology to a non -\ntechnical industry  \n•Geographic focus  is either \nwholly or partially in a startup \nhub \n•Hiring needs are primarily \nsales and marketing team \nmembers  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a',
    'page number': 'Page16'},
   {'chunk': 1,
    'text': 'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP',
    'page number': 'Page16'},
   {'chunk': 0,
    'text': '2. A Typical Distribution Schedule  \nSeniority  Equity Allocation  \nFirst 10 Employees  10% \nNext 20 Employees  5% \nNext 50 Employees  5% \n•Early -stage equity grants are always a negotiation, but generally:  \n–CTO: 1 -5% \n–Key Developer or Engineer: 1 -2% \n–Other Functional Team Member: 0.5 -1.5%  \n–No non -founding member of the senior team should receive over 10%',
    'page number': 'Page17'},
   {'chunk': 0,
    'text': 'The Bottom -Up Process  \n1.Segment your human resources  \n2.Create pay multipliers for each job function  \n3.Determine the dollar value of an options grant  \n4.Determine the current share price  \n5.Calculate options grants',
    'page number': 'Page18'},
   {'chunk': 0,
    'text': '1. Segment Your Human Resources  \nSegment  Roles  Award  \nin % or $?  \nFounders  Co-Founders  % \nFounding Team  Technical & Product  % \nEarly -Stage Hires  Senior Team  (C-Level)  $ \nDirectors / VPs  $ \nFunctional / Technical Team  $ \nSupport Team  $ \n•Think about the early -stage team in three segments  \n–Founders : the founding partners in the business  \n–Founding Team : the people you hire to build your product  \n–Early -Stage Hires : the team you build as you raise capital  \n•As early as possible, begin communicating options grants in terms of \ndollars rather than percentages',
    'page number': 'Page19'},
   {'chunk': 0,
    'text': '2. Establish Pay Multipliers for Each Role  \nSegment  Roles  Award  \nin % or $?  Options Multiplier  \n(Multiple  of Salary)  \nFounders  Co-Founders  % -- \nFounding Team  Technical & Product  % Negotiated  \nEarly -Stage Hires  Senior Team  (C-Level)  $ 0.5x – 1.0x base  salary  \nDirectors / VPs  $ 0.25x – 0.5x base salary  \nFunctional / Technical Team  $ 0.1x – 0.25x base  salary  \nSupport Team  $ 0.0x – 0.1x base salary  \n•After segmenting the early -stage team, establish a standardized \nmultiple of base salary to be granted as an option package  \n–These ranges provide a ballpark grant for each role  \n–Get away from actively negotiating packages with employees  \n–Be transparent about this methodology  \nSource: This range of multiples was provided by Fred Wilson of Union Square Ventures, \nwhich he created in collaboration with a leading HR consulting firm.',
    'page number': 'Page20'},
   {'chunk': 0,
    'text': '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule',
    'page number': 'Page21'},
   {'chunk': 0,
    'text': '4. Determine the Current Share Price  \nLatest “True” Valuation  \nFully -Diluted Share Count  Current Share Price  = What is the company actually worth today?  Or, at what \nvaluation could you raise capital or sell the company? This \nis an internal valuation (not published). Things to consider:  \n-The latest financing round  \n-Any bonafide  acquisition offer  \n-Comparable company valuations / DCF analysis  \nHow many shares are outstanding?  This should include \nall shares issued, as well as any dilutive securities:  \n-Common shares  \n-Preferred shares  \n-Shares authorized for issuance  \n-Options or warrants',
    'page number': 'Page22'},
   {'chunk': 0,
    'text': '5. Calculate the Options Grant  \nOptions Grant  \n(Number of Shares)  = Dollar Value  \nof Options Grant  \nCurrent Share Price',
    'page number': 'Page23'},
   {'chunk': 0,
    'text': '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS',
    'page number': 'Page24'},
   {'chunk': 0,
    'text': 'Important Takeaways  \n•Top-down planning (“the allocation”)  \n–Holistically consider what percentage of the company should \nbelong to employees  \n–Allocate these shares to an ESOP  \n•Bottom -up planning (“the plan”)  \n–Specifically consider how much equity each employee should be \nawarded  \n–Use this framework for individual equity grants  \n•Key Takeaway : Get away from ad hoc equity awards and personal \nnegotiations by standardizing both the amount of equity available to \nemployees and the process by which packages are awarded',
    'page number': 'Page25'},
   {'chunk': 0,
    'text': 'THE FINE PRINT – TERMS  Part III',
    'page number': 'Page26'},
   {'chunk': 0,
    'text': 'Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”',
    'page number': 'Page27'},
   {'chunk': 0,
    'text': 'Vesting Schedule  \n•Vesting protects the company  \n–It stages the accrual of options, mitigating the risk that an employee \nwill depart with an undeserved equity stake  \n–Continually incentivizes employees as they earn their options \npackage over the course of the vesting period  \n•Standard vesting period: 4 years  \n–Alternatively, some companies offer 3 -year vesting, although with a \nshorter vesting period they may offer a smaller options package  The vesting schedule is the timetable over which an employee accrues the \nright to keep the options that have been awarded',
    'page number': 'Page28'},
   {'chunk': 0,
    'text': 'The Cliff  \n•The cliff protects the company  \n•This trial period exists so that the company can avoid giving shares to \nemployees immediately, reserving the opportunity part ways if the \nemployee does not perform well  \n•Standard cliff: 1 year  \n–Alternatively, some companies require a 2 -year cliff. However, in \nstartup communities where employees know what is “standard” this \ncan cause significant discontent  \n–Firing an employee right as their cliff is about to expire is legal, but \nunethical – begin to communicate discontent as early as possible \nduring the trail period  The trial period during which no vesting occurs; in this period vesting \naccrues, but the accrued shares are earned immediately after the cliff',
    'page number': 'Page29'},
   {'chunk': 0,
    'text': '200 shares400 shares600 shares800 shares1,000 shares\n0 months 12 months 24 months 36 months 48 monthsExample: Standard Vesting with a Cliff  \n•New hire  \n•Granted options for 1,000 shares  \n•4-year vesting schedule with 1 -year cliff  \n–At 12 months earns 250 shares  \n–Then earns ~21 shares per month  \nMonths on the Job  Vested Options  \nThe cliff  Monthly vesting',
    'page number': 'Page30'},
   {'chunk': 0,
    'text': 'Vesting in a Liquidity Event  \nThe terms of an options package should specifically address what \nhappens in a liquidity event such as an IPO or acquisition  \n•Vested options generally become exercisable  \n•Unvested options can either:  \n–Roll over and remain unvested  \n–Receive accelerated vesting  \n•These options vest automatically upon change of control – not \ncommon for everyone, but typical for top employees  \n–Receive double -trigger vesting  \n•These options vest automatically upon change of control, but \nonly if the employee is terminated – not common for everyone, \nbut typical for employees who may be extraneous in a merger \n(CFO, General Counsel)',
    'page number': 'Page31'},
   {'chunk': 0,
    'text': 'Exercising Options  \n•Options are “in the money” if the share value is above the strike price  \n•To exercise an option, the employee pays the strike price to the \ncompany and receives shares for each option exercised  \n•If there has been a liquidity event  \n–In the case of an IPO , the employee can now sell these shares in \nthe public markets  \n–In the case of a cash acquisition , an employee’s shares are \nbought out at the cash offer price  \n–In the case of a stock acquisition , an employee’s shares are \nconverted to shares in the new company  Options are usually exercised following a liquidity event, but employees \ncan exercise them at any time after they vest',
    'page number': 'Page32'},
   {'chunk': 0,
    'text': 'Tax Considerations  \nOptions can have material tax consequences for employees; startups \nneed to understand the tax implications and be upfront with new hires  \n•Tax treatment of options will vary by region  \n•Tax treatment will vary by the type option security issued  \n•Several possibilities:  \n–Taxable event upon grant  \n–Taxable event upon leaving company  \n–Taxable event upon exercise  \n–No effective tax event  \n•It is important for employees to fully understand the tax implications of \nthe securities they are receiving',
    'page number': 'Page33'},
   {'chunk': 0,
    'text': 'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language',
    'page number': 'Page34'},
   {'chunk': 0,
    'text': 'ESOPS FOR THE LONG TERM  Part IV',
    'page number': 'Page35'},
   {'chunk': 0,
    'text': 'Retention Grants  \n•Retention grants are used to incentive employees to stay with the \ncompany beyond their 4 -year vesting period: a “top off”  \n–Do not wait until an employee is fully vested to make a retention \ngrant! This creates a natural exit incentive at the end of 4 years  \n–Instead, consider making smaller retention grants every 2 years  \n•A framework for retention grants:  \nGrant Date  Every 2  years, beginning 2 years after initial package  \nOptions Multiplier:  Use ½ of the employee’s standard options multiplier  \nSalary Baseline:  Use current salary  if employee has received a raise   \nValuation & Shares  Use current valuation and share count, if changed  \nRetention  \nGrant  = Current \nSalary  New (Latest) Valuation  \nDiluted Share Count  x 50% of Options \nMultiplier  ÷',
    'page number': 'Page36'},
   {'chunk': 0,
    'text': 'Discretionary Grants  \n•Bonuses Outstanding Performance  \n–These discretionary grants should only be made once per year, and \nonly offered to your top 10 -20% of employees whose \naccomplishments and performance are truly distinguished  \n•Promotions  \n–These discretionary grants should only be made to reward \npromotion. Bring the employee’s total equity up to the amount you \nwould pay to hire them today.',
    'page number': 'Page37'},
   {'chunk': 0,
    'text': 'Social Impact Considerations  \n•Companies focused on social impact goals and/or in developing \nmarkets may have unique ESOP considerations  \n•Consider two possible scenarios:  \n–Financial Inclusion Goals : Employee -friendly hiring practices and \nESOPs can help build local financial inclusion. If this is part of your \ncorporate mandate, consider the added social impact value of \noffering your options program to all levels of employees  \n–Local Ownership Culture : Conversely, in certain regions stock \noptions may have negligible value to employees, either because of \nrisk aversion, lack of liquidity, or lack of understanding. If this is the \ncase, it is not worth extensively offering options to employees who \nwould rather be paid in cash',
    'page number': 'Page38'},
   {'chunk': 0,
    'text': 'Options Modeling – Overview  \n•Creating an ESOP has lasting implications on startup capital structure  \n•While the guidelines provided in this presentation offer a roadmap to \nimplementing an options plan, it can also be helpful to build an options \nmodel to understand the long -term effects of this program  \n•Options models:  \n–Forecast hiring needs over time  \n–Estimate future valuation and fundraising needs  \n–Provide an estimate of employee ownership  \nNote: There is an accompanying excel file that provides a template for \nbuilding an options model similar to that described on the following slide',
    'page number': 'Page39'},
   {'chunk': 0,
    'text': 'Options Modeling – A Detailed Example  \nEmployee Stock Options Plan (ESOP) Model\nYear 0 Year 1 Year 2 Year 3 Year 4\nValue Shares Value Shares Shr. Price Value Shares Shr. Price Value Shares Shr. Price\nNA 10,000 $3M 10,000 $300 $5M 10,000 $500 $15M 12,000 $1,250\nOptions Total Equity Grant Total Equity Grant Total Equity Grant Total Equity\nSalary Multiplier Options (%) Amount Options (%) Amount Options (%) Amount Options (%)\nTotal Non-Founder Employee Ownership 6.50% 9.00% 11.64% 10.20%\nFounding Team\nCTO $120,000 1.00x 400 4.00% 400 4.00% $60,000 520 5.20% 520 4.33%\nProduct Manager $80,000 0.80x 250 2.50% 250 2.50% $32,000 314 3.14% 314 2.62%\nYear 1 Hires\nLead Engineer $80,000 0.50x $40,000 133 1.33% 133 1.33% $20,000 149 1.24%\nLead Designer $70,000 0.50x $35,000 117 1.17% 117 1.17% $17,500 131 1.09%\nYear 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires',
    'page number': 'Page40'},
   {'chunk': 1,
    'text': 'Year 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires\nMarketing Director $60,000 0.30x $18,000 14 0.12%\nEngineer $50,000 0.25x $12,500 10 0.08%\nSalesperson $40,000 0.10x $4,000 3 0.03%\nOffice Manager $30,000 0.10x $3,000 2 0.02%\nFounding team shares are \nnegotiated as a percentage of the \ncompany; all future employees \nreceive grants determined in dollars  \nRetention grants are awarded every two \nyears after an initial options package  The “true” valuation (an \ninternal estimate) increases \nmarginally as the company \nbuilds value…  …but increases significantly when \nthe company raises a new round, \nwhich also increases the share \ncount and dilutes all equity owners  \nFor all new hires after the \nfounding team, options grants \nare awarded based on the \nemployee’s options multiplier \nand baseline salary',
    'page number': 'Page40'},
   {'chunk': 0,
    'text': 'RESOURCES & FURTHER \nREADING  Part V',
    'page number': 'Page41'},
   {'chunk': 0,
    'text': 'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity',
    'page number': 'Page42'},
   {'chunk': 0,
    'text': 'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting',
    'page number': 'Page43'},
   {'chunk': 0,
    'text': 'This work  is licensed  under  the Creative  Commons  Attribution -\nShareAlike  4.0 International  License . To view a copy  of this license,  \nvisit http://creativecommons .org/licenses/by -sa/4.0/ or send  a letter  to \nCreative  Commons,  PO Box 1866 , Mountain  View,  CA 94042 , USA.',
    'page number': 'Page44'}],
  ['what is a neccessary Part of the Capital Structure according to Fred Wilson?',
   'When to Communicate Options to Employees: % versus $',
   'When to Create an ESOP?',
   'What are the Common Terms in an Options Package?']),
 HuggingFaceEmbeddings(client=SentenceTransformer(
   (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
   (2): Normalize()), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}))

SyntaxError: unterminated string literal (detected at line 28) (<ipython-input-21-f7f1fbc23446>, line 28)

In [7]:
#Convert Scanned Document to Image
def convert_pdf_to_image(pdf_path):
    pages = convert_from_path(pdf_path, 350)
    #
    output_dir="pdf_img_dir"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"
        image_path = os.path.join(output_dir,image_name)
        page.save(image_path, "JPEG")
        i = i+1
    return output_dir

In [8]:
# Extract text from image
#
def extract_text_from_image(image_path):
    extractedInformation = pytesseract.image_to_string(Image.open(image_path))
    return extractedInformation
#

In [9]:
# Compute Cosine Similarity
def get_scores(sentence1,sentence2,smodel):
  # encode sentences to get their embeddings
  embedding1 = smodel.encode(sentence1, convert_to_tensor=True)
  embedding2 = smodel.encode(sentence2, convert_to_tensor=True)
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  return cosine_scores.numpy().tolist()[0][0]
#
#Text Chuncking
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
                            chunk_size=1000,
                            chunk_overlap=200,
                            separators=["\n\n", "\n", " ", ""]
                            )
    chunks = text_splitter.split_text(text)
    return chunks
#
# Data preprocessing step
def preprocess_function(request_data,content_type=None):
  pdf_docs = request_data["pdf1"]
  prompt = request_data["query"]
  flag,page_num,page_content = is_scanned_pdf(pdf_docs)
  pdf_name = "temppdf"
  f_path = os.path.join("/tmp", pdf_name)
  #
  print(pdf_docs)
  if flag:
    with open(f_path, "wb") as f:
        f.write(pdf_docs.getbuffer())
    print(f"filesize : {os.path.getsize(f_path)}")
    page_num  = []
    page_content = []
    text = ""
    # convert pdf to image
    image_dir = convert_pdf_to_image(f_path)
    for i,file in enumerate(os.listdir(image_dir)):
      page_num.append(f"Page{i+1}")
      image_path = os.path.join(image_dir,file)
      text += extract_text_from_image(image_path)
      page_content.append(extract_text_from_image(image_path))

      print(f"Text read from image: {text}")
  #
  metadatas = []
  texts = []
  for p,record in tqdm(zip(page_num,page_content)):
        # first get metadata fields for this record
        metadata = {'page number': p}
        # now we create chunks from the record text
        record_texts = get_text_chunks(record)
        # create individual metadata dicts for each chunk
        record_metadatas = [{
            "chunk": j, "text": text, **metadata
        } for j, text in enumerate(record_texts)]
        # append these to current batches
        texts.extend(record_texts)
        metadatas.extend(record_metadatas)
  c = []
  for i in range(len(metadatas)):
    c.append(metadatas[i]['text'])

  page_content = c
  return (flag,page_num,page_content,metadatas,prompt)
#
# Load the required embeddings
def model_load_function(model_path):
  embeddings = HuggingFaceEmbeddings(model_kwargs = {'device': 'cpu'},
                                    encode_kwargs = {'normalize_embeddings': False})
  smodel = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
  #
  return (embeddings,model_path,smodel)
#
# The Prediction function , it returns the response as well as the embeddinbgs used
def predict_function(context,model,content_type=None):
  embeddings,model_path,smodel= model
  flag,page_num,page_content,metadatas,prompt = context
  #create vector store
  #

  doc_store = Qdrant.from_texts(page_content,
                              metadatas=metadatas,
                              embedding=embeddings,
                              location=":memory:",
                              prefer_grpc=True,

                              collection="doc_search")

  #query vector store
  prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
  Please provide an answer which is factually correct and based on the information retrieved from the vector store.
  Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
  """
  PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
  )
  #
  chain_type_kwargs = {"prompt": PROMPT}
  #
  qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo-16k',
                                                  openai_api_key=os.environ["OPENAI_API_KEY"],
                                                  temperature=0),
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt": PROMPT},
                                  retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                      return_source_documents=True
                                  )



  # Serach for answer
  print(prompt)
  final_result = []
  for pmpt in prompt:
    result = qa(pmpt)
    final_result.append(result)
  return (final_result,smodel,context,embeddings)
#
#Processing the Response
#
def postprocess_function(res, content_type=None):
  results,smodel,context,embeddings = res
  flag,page_num,page_content,metadatas,prompt = context
  #
  queries =[]
  answers = []
  final_ref = []
  final_pages = []
  final_scores = []
  for result in results :
    query = result['query']
    answer = result['result']
    queries.append(query)
    answers.append(answer)
    referrences = []
    pages = []
    scores = []
    #
    for i in range(len(result["source_documents"])):
      print(f"Content : {result['source_documents'][i].page_content}")
      referrences.append(result['source_documents'][i].page_content)
      print(f"Page Number : {result['source_documents'][i].metadata['page number']}")
      pages.append(result['source_documents'][i].metadata['page number'])

    final_ref.append(referrences)
    final_pages.append(pages)
    # Get the similarity scores for the documents selected

    for item in referrences:
      scores.append(get_scores(query,item,smodel))
    final_scores.append(scores)
  # Format the response
  predictions = {"Query":queries,
                 "Answer":answers,
                 "Referrences":final_ref,
                 "Page Numbers":final_pages,
                 "Scores": final_scores}
  #Generate Database dataframe
  t = []
  e = []
  for ts in page_content:
    t.append(ts)
    e.append(embeddings.embed_documents(ts)[0])
  database_df = pd.DataFrame({"text":t,"text_vector":e})
  #database_df.to_parquet("database_df.parquet",index=False)
  #processing quer_df attributes
  txt = []
  text_vector = []
  response = []
  context_text_0 = []
  context_text_1 = []
  context_similarity_0 = []
  context_similarity_1 = []
  user_feedback = []

  for q,a,refs,scrs in zip(queries,answers,final_ref,final_scores):
    txt.append(q)
    text_vector.append(embeddings.embed_query(q))
    response.append(a)
    context_text_0.append(refs[0])
    context_text_1.append(refs[1])
    context_similarity_0.append(scrs[0])
    context_similarity_1.append(scrs[1])
    user_feedback.append(1.0)
  #Create query dataframe based on the response received
  query_df = pd.DataFrame({"text":txt,
                           "text_vector":text_vector,
                           "response":response,
                           "context_text_0":context_text_0,
                           "context_text_1":context_text_1,
                           "context_similarity_0":context_similarity_0,
                           "context_similarity_1":context_similarity_1,
                           "user_feedback":user_feedback})
  # delete temp directories if any created
  output_dir="pdf_img_dir"
  if os.path.exists(output_dir):
    for  file in os.listdir(output_dir):
      file_name = os.path.join(output_dir,file)
      if os.path.exists(file_name) and os.path.isfile(file_name):
        print(f"File {file_name} exists. Hence Removing")
        os.remove(file_name)
  return (predictions,database_df,query_df)

In [22]:
%%time
# We have passes a set of queries corresponding to the pdf uploaded
request_data = {"pdf1":"/content/policy-booklet-0923.pdf",
                "query":["what is a neccessary Part of the Capital Structure according to Fred Wilson?",
                         "When to Communicate Options to Employees: % versus $",
                         "When to Create an ESOP?",
                         "What are the Common Terms in an Options Package?"]}
content = preprocess_function(request_data,content_type=None)

Text-based PDF
/content/63fddf948f3f2e61e9b9afbe_India ESOP Guide from Cake Equity.pdf


0it [00:00, ?it/s]

CPU times: user 474 ms, sys: 12.5 ms, total: 486 ms
Wall time: 506 ms


In [11]:
%%time
model = model_load_function(model_path=None)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 1.66 s, sys: 1.76 s, total: 3.42 s
Wall time: 15.9 s


In [23]:
%%time
response = predict_function(content,model,content_type=None)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


['what is a neccessary Part of the Capital Structure according to Fred Wilson?', 'When to Communicate Options to Employees: % versus $', 'When to Create an ESOP?', 'What are the Common Terms in an Options Package?']
CPU times: user 32 s, sys: 5.76 s, total: 37.7 s
Wall time: 44 s


In [24]:
#response
([{'query': 'what is a neccessary Part of the Capital Structure according to Fred Wilson?',
   'result': 'According to Fred Wilson, a necessary part of the capital structure is employee equity or ESOPs (Employee Stock Ownership Plans). He states, "I can\'t think of a term sheet that we have issued that didn\'t have a specific provision for employee equity." This suggests that offering employee equity is a prerequisite for venture capitalists and is commonly included in deals with startups.',
   'source_documents': [Document(page_content="A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures", metadata={'chunk': 0, 'text': "A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures", 'page number': 'Page10'}),
    Document(page_content='The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time', metadata={'chunk': 0, 'text': 'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time', 'page number': 'Page15'}),
    Document(page_content='Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity', metadata={'chunk': 0, 'text': 'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity', 'page number': 'Page42'}),
    Document(page_content='Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting', metadata={'chunk': 0, 'text': 'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting', 'page number': 'Page43'}),
    Document(page_content='Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage', metadata={'chunk': 1, 'text': 'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage', 'page number': 'Page11'})]},
  {'query': 'When to Communicate Options to Employees: % versus $',
   'result': 'The context does not provide information about when to communicate options to employees as a percentage versus a dollar value.',
   'source_documents': [Document(page_content='Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather', metadata={'chunk': 0, 'text': 'Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather', 'page number': 'Page12'}),
    Document(page_content='value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', metadata={'chunk': 1, 'text': 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', 'page number': 'Page12'}),
    Document(page_content='Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success', metadata={'chunk': 0, 'text': 'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success', 'page number': 'Page8'}),
    Document(page_content='0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS', metadata={'chunk': 0, 'text': '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS', 'page number': 'Page24'}),
    Document(page_content='3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule', metadata={'chunk': 0, 'text': '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule', 'page number': 'Page21'})]},
  {'query': 'When to Create an ESOP?',
   'result': 'ESOPs should be created during the seed stage or early-VC stage of a startup.',
   'source_documents': [Document(page_content='When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted', metadata={'chunk': 0, 'text': 'When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted', 'page number': 'Page11'}),
    Document(page_content='Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired', metadata={'chunk': 0, 'text': 'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired', 'page number': 'Page14'}),
    Document(page_content='members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP', metadata={'chunk': 1, 'text': 'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP', 'page number': 'Page16'}),
    Document(page_content='Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language', metadata={'chunk': 0, 'text': 'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language', 'page number': 'Page34'}),
    Document(page_content='ESOPS FOR THE LONG TERM  Part IV', metadata={'chunk': 0, 'text': 'ESOPS FOR THE LONG TERM  Part IV', 'page number': 'Page35'})]},
  {'query': 'What are the Common Terms in an Options Package?',
   'result': 'The common terms in an options package are:\n1. Number of Shares: The total number of options granted to an employee, and the maximum number of shares that employee has access to.\n2. Strike Price: The price the employee must pay to purchase each share if and when the employee chooses to exercise the option.\n3. Vesting Schedule: The timeline over which the options become wholly owned and exercisable by the employee, no longer subject to repurchase by the company.\n4. Cliff Period: The trial period during which no vesting occurs; in this period vesting accrues, but the total effect of this vesting is realized immediately after the cliff.\n5. Expiration Date: The last date on which the options may be exercised and converted into common shares by the employee.\n\nQuotes supporting the answer:\n- "Number of Shares: The total number of options granted to an employee, and therefore the maximum number of shares that employee has access to."\n- "Strike Price: The price the employee must pay to purchase each share if and when the employee chooses to exercise the option."\n- "Vesting Schedule: The timeline over which the options become wholly owned and exercisable by the employee (no longer subject to repurchase by the company)."\n- "Cliff Period: The trial period during which no vesting occurs; in this period vesting accrues, but the total effect of this vesting is realized immediately after the cliff."\n- "Expiration Date: The last date on which the options may be exercised and converted into common shares by the employee."',
   'source_documents': [Document(page_content='Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants', metadata={'chunk': 0, 'text': 'Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants', 'page number': 'Page7'}),
    Document(page_content='Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time', metadata={'chunk': 1, 'text': 'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time', 'page number': 'Page5'}),
    Document(page_content='What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity', metadata={'chunk': 0, 'text': 'What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity', 'page number': 'Page5'}),
    Document(page_content='value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', metadata={'chunk': 1, 'text': 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', 'page number': 'Page12'}),
    Document(page_content='Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”', metadata={'chunk': 0, 'text': 'Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”', 'page number': 'Page27'})]}],
 SentenceTransformer(
   (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
   (2): Normalize()
 ),
 (False,
  ['Page1',
   'Page2',
   'Page3',
   'Page4',
   'Page5',
   'Page6',
   'Page7',
   'Page8',
   'Page9',
   'Page10',
   'Page11',
   'Page12',
   'Page13',
   'Page14',
   'Page15',
   'Page16',
   'Page17',
   'Page18',
   'Page19',
   'Page20',
   'Page21',
   'Page22',
   'Page23',
   'Page24',
   'Page25',
   'Page26',
   'Page27',
   'Page28',
   'Page29',
   'Page30',
   'Page31',
   'Page32',
   'Page33',
   'Page34',
   'Page35',
   'Page36',
   'Page37',
   'Page38',
   'Page39',
   'Page40',
   'Page41',
   'Page42',
   'Page43',
   'Page44'],
  ['Startup Employee Stock \nOptions Plans (ESOPs)  \n \nOverview and Best Practices',
   'Table of Contents  \nPart I: Intro to Options Plans  \n•What is an ESOP?  \n•What is an Option?  \n•Lifecycle of a Startup ESOP  \n•Common Terms in an Options Package  \n•Why Issue Options to Employees ? \n–A Defining Characteristic of Startup Culture  \n–A Necessary Part of the Capital Structure  \n•When to Create an ESOP ? \n•Communicating Options to Employees: % versus $ \n \nPart II: How Much to Grant  \n•Two Approaches  \n•The Top -Down Process  \n–1. How Much Equity to Set Aside in the ESOP ? \n–2. A Typical Distribution Schedule  \n•The Bottom -Up Process  \n–1. Segment Your Human Resources  \n–2. Establish Pay Multipliers for Each Role \n–3. Determine the Dollar Value of the Options Grant  \n–4. Determine the Current Share Price  \n–5. Calculate the Options Grant  \n–An Example: Hiring a CTO  \n•Important Takeaways  Part III : The Fine Print – Terms  \n•Strike Price  \n•Vesting Schedule  \n•The Cliff  \n–Example: Standard Vesting w/ a Cliff \n•Vesting in a Liquidity Event  \n•Exercising Options',
   '•Important Takeaways  Part III : The Fine Print – Terms  \n•Strike Price  \n•Vesting Schedule  \n•The Cliff  \n–Example: Standard Vesting w/ a Cliff \n•Vesting in a Liquidity Event  \n•Exercising Options  \n•Tax Considerations  \n•Legal Advice  \n \nPart IV: ESOPs for the Long Term  \n•Retention Grants  \n•Discretionary Grants  \n•Social Impact Considerations  \n•Options Modeling – Overview  \n•Options Modeling – A Detailed Example  \n \nPart V: Resources & Further Reading',
   'INTRO TO OPTIONS PLANS  Part I',
   'What is an ESOP?  \n•An Employee Stock Options Plan (ESOP)  \n \n•An allocation  of shares that will be granted to employees in the future \nin the form of stock options  \n–How much equity should we set aside for employees?  \n \n•A plan for how these options will be distributed:  \n–How many shares will individual employees receive?  \n–What terms will govern these grants?  \n \n•The plan is as important as the allocation!',
   'What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity',
   'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time',
   'Lifecycle of a Startup ESOP  \n•Founders and early investors  create an ESOP by setting aside a \npercentage of shares to be granted to future employees  \n•Management and the Board of Directors  issue these shares to \nemployees as options packages granted for hiring, promotion and \nretention  \n•Employees  receive all of their options upfront, but the company \nmaintains a right to force forfeit that diminishes over time through a \nprocess called “vesting”  \n•Options are exercised  by employees when the company is acquired \nor taken public. The employee pays the “strike price” to acquire the \nshares, but those shares are now marketable at a higher value',
   'Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants',
   'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success',
   'The defining difference between Silicon Valley companies and almost \nevery other industry in the U.S. is the virtually universal practice among \ntech companies of distributing meaningful equity (usually in the form of \nstock options) to ordinary employees .  A Defining Characteristic of Startup Culture  \nSteven Johnson, Technology Writer  \n•Startups are a unique case . Unlike at larger corporations, employee \nownership is an essential element of startup communities and culture  \n–As high -risk/high -reward enterprises, startups use options to align \nemployee compensation with the risk -prone mentality of the \nbusiness  \n–Startups seeking to achieve a “big exit” use options to align all \nemployees to drive toward this desired outcome',
   "A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures",
   'When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted',
   'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage',
   'Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather',
   'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .',
   'HOW MUCH TO GRANT  Part II',
   'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired',
   'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time',
   '1. How Much Equity to Set Aside in the ESOP?  \n•Founders  are top managers \nwith functional roles that are \nnot redundant  \n•DNA  of the company is in a \nnon-technical industry; no \ntechnology emphasis  \n•Geographic focus  is on a \nregion that is not considered a \nstartup hub  \n•Hiring needs are primarily \nadministrative or back -office \nteam members  <15%  20% 25% Generally non-founder employees  get 15 -20% of the company, with some \ncompanies issuing up to 25% (and a current trend toward bigger ESOPs)  \n•Founders  are top mangers, \nbut hire others to fill key \ntechnical or functional roles  \n•DNA  of the company is in \napplying technology to a non -\ntechnical industry  \n•Geographic focus  is either \nwholly or partially in a startup \nhub \n•Hiring needs are primarily \nsales and marketing team \nmembers  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a',
   'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP',
   '2. A Typical Distribution Schedule  \nSeniority  Equity Allocation  \nFirst 10 Employees  10% \nNext 20 Employees  5% \nNext 50 Employees  5% \n•Early -stage equity grants are always a negotiation, but generally:  \n–CTO: 1 -5% \n–Key Developer or Engineer: 1 -2% \n–Other Functional Team Member: 0.5 -1.5%  \n–No non -founding member of the senior team should receive over 10%',
   'The Bottom -Up Process  \n1.Segment your human resources  \n2.Create pay multipliers for each job function  \n3.Determine the dollar value of an options grant  \n4.Determine the current share price  \n5.Calculate options grants',
   '1. Segment Your Human Resources  \nSegment  Roles  Award  \nin % or $?  \nFounders  Co-Founders  % \nFounding Team  Technical & Product  % \nEarly -Stage Hires  Senior Team  (C-Level)  $ \nDirectors / VPs  $ \nFunctional / Technical Team  $ \nSupport Team  $ \n•Think about the early -stage team in three segments  \n–Founders : the founding partners in the business  \n–Founding Team : the people you hire to build your product  \n–Early -Stage Hires : the team you build as you raise capital  \n•As early as possible, begin communicating options grants in terms of \ndollars rather than percentages',
   '2. Establish Pay Multipliers for Each Role  \nSegment  Roles  Award  \nin % or $?  Options Multiplier  \n(Multiple  of Salary)  \nFounders  Co-Founders  % -- \nFounding Team  Technical & Product  % Negotiated  \nEarly -Stage Hires  Senior Team  (C-Level)  $ 0.5x – 1.0x base  salary  \nDirectors / VPs  $ 0.25x – 0.5x base salary  \nFunctional / Technical Team  $ 0.1x – 0.25x base  salary  \nSupport Team  $ 0.0x – 0.1x base salary  \n•After segmenting the early -stage team, establish a standardized \nmultiple of base salary to be granted as an option package  \n–These ranges provide a ballpark grant for each role  \n–Get away from actively negotiating packages with employees  \n–Be transparent about this methodology  \nSource: This range of multiples was provided by Fred Wilson of Union Square Ventures, \nwhich he created in collaboration with a leading HR consulting firm.',
   '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule',
   '4. Determine the Current Share Price  \nLatest “True” Valuation  \nFully -Diluted Share Count  Current Share Price  = What is the company actually worth today?  Or, at what \nvaluation could you raise capital or sell the company? This \nis an internal valuation (not published). Things to consider:  \n-The latest financing round  \n-Any bonafide  acquisition offer  \n-Comparable company valuations / DCF analysis  \nHow many shares are outstanding?  This should include \nall shares issued, as well as any dilutive securities:  \n-Common shares  \n-Preferred shares  \n-Shares authorized for issuance  \n-Options or warrants',
   '5. Calculate the Options Grant  \nOptions Grant  \n(Number of Shares)  = Dollar Value  \nof Options Grant  \nCurrent Share Price',
   '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS',
   'Important Takeaways  \n•Top-down planning (“the allocation”)  \n–Holistically consider what percentage of the company should \nbelong to employees  \n–Allocate these shares to an ESOP  \n•Bottom -up planning (“the plan”)  \n–Specifically consider how much equity each employee should be \nawarded  \n–Use this framework for individual equity grants  \n•Key Takeaway : Get away from ad hoc equity awards and personal \nnegotiations by standardizing both the amount of equity available to \nemployees and the process by which packages are awarded',
   'THE FINE PRINT – TERMS  Part III',
   'Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”',
   'Vesting Schedule  \n•Vesting protects the company  \n–It stages the accrual of options, mitigating the risk that an employee \nwill depart with an undeserved equity stake  \n–Continually incentivizes employees as they earn their options \npackage over the course of the vesting period  \n•Standard vesting period: 4 years  \n–Alternatively, some companies offer 3 -year vesting, although with a \nshorter vesting period they may offer a smaller options package  The vesting schedule is the timetable over which an employee accrues the \nright to keep the options that have been awarded',
   'The Cliff  \n•The cliff protects the company  \n•This trial period exists so that the company can avoid giving shares to \nemployees immediately, reserving the opportunity part ways if the \nemployee does not perform well  \n•Standard cliff: 1 year  \n–Alternatively, some companies require a 2 -year cliff. However, in \nstartup communities where employees know what is “standard” this \ncan cause significant discontent  \n–Firing an employee right as their cliff is about to expire is legal, but \nunethical – begin to communicate discontent as early as possible \nduring the trail period  The trial period during which no vesting occurs; in this period vesting \naccrues, but the accrued shares are earned immediately after the cliff',
   '200 shares400 shares600 shares800 shares1,000 shares\n0 months 12 months 24 months 36 months 48 monthsExample: Standard Vesting with a Cliff  \n•New hire  \n•Granted options for 1,000 shares  \n•4-year vesting schedule with 1 -year cliff  \n–At 12 months earns 250 shares  \n–Then earns ~21 shares per month  \nMonths on the Job  Vested Options  \nThe cliff  Monthly vesting',
   'Vesting in a Liquidity Event  \nThe terms of an options package should specifically address what \nhappens in a liquidity event such as an IPO or acquisition  \n•Vested options generally become exercisable  \n•Unvested options can either:  \n–Roll over and remain unvested  \n–Receive accelerated vesting  \n•These options vest automatically upon change of control – not \ncommon for everyone, but typical for top employees  \n–Receive double -trigger vesting  \n•These options vest automatically upon change of control, but \nonly if the employee is terminated – not common for everyone, \nbut typical for employees who may be extraneous in a merger \n(CFO, General Counsel)',
   'Exercising Options  \n•Options are “in the money” if the share value is above the strike price  \n•To exercise an option, the employee pays the strike price to the \ncompany and receives shares for each option exercised  \n•If there has been a liquidity event  \n–In the case of an IPO , the employee can now sell these shares in \nthe public markets  \n–In the case of a cash acquisition , an employee’s shares are \nbought out at the cash offer price  \n–In the case of a stock acquisition , an employee’s shares are \nconverted to shares in the new company  Options are usually exercised following a liquidity event, but employees \ncan exercise them at any time after they vest',
   'Tax Considerations  \nOptions can have material tax consequences for employees; startups \nneed to understand the tax implications and be upfront with new hires  \n•Tax treatment of options will vary by region  \n•Tax treatment will vary by the type option security issued  \n•Several possibilities:  \n–Taxable event upon grant  \n–Taxable event upon leaving company  \n–Taxable event upon exercise  \n–No effective tax event  \n•It is important for employees to fully understand the tax implications of \nthe securities they are receiving',
   'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language',
   'ESOPS FOR THE LONG TERM  Part IV',
   'Retention Grants  \n•Retention grants are used to incentive employees to stay with the \ncompany beyond their 4 -year vesting period: a “top off”  \n–Do not wait until an employee is fully vested to make a retention \ngrant! This creates a natural exit incentive at the end of 4 years  \n–Instead, consider making smaller retention grants every 2 years  \n•A framework for retention grants:  \nGrant Date  Every 2  years, beginning 2 years after initial package  \nOptions Multiplier:  Use ½ of the employee’s standard options multiplier  \nSalary Baseline:  Use current salary  if employee has received a raise   \nValuation & Shares  Use current valuation and share count, if changed  \nRetention  \nGrant  = Current \nSalary  New (Latest) Valuation  \nDiluted Share Count  x 50% of Options \nMultiplier  ÷',
   'Discretionary Grants  \n•Bonuses Outstanding Performance  \n–These discretionary grants should only be made once per year, and \nonly offered to your top 10 -20% of employees whose \naccomplishments and performance are truly distinguished  \n•Promotions  \n–These discretionary grants should only be made to reward \npromotion. Bring the employee’s total equity up to the amount you \nwould pay to hire them today.',
   'Social Impact Considerations  \n•Companies focused on social impact goals and/or in developing \nmarkets may have unique ESOP considerations  \n•Consider two possible scenarios:  \n–Financial Inclusion Goals : Employee -friendly hiring practices and \nESOPs can help build local financial inclusion. If this is part of your \ncorporate mandate, consider the added social impact value of \noffering your options program to all levels of employees  \n–Local Ownership Culture : Conversely, in certain regions stock \noptions may have negligible value to employees, either because of \nrisk aversion, lack of liquidity, or lack of understanding. If this is the \ncase, it is not worth extensively offering options to employees who \nwould rather be paid in cash',
   'Options Modeling – Overview  \n•Creating an ESOP has lasting implications on startup capital structure  \n•While the guidelines provided in this presentation offer a roadmap to \nimplementing an options plan, it can also be helpful to build an options \nmodel to understand the long -term effects of this program  \n•Options models:  \n–Forecast hiring needs over time  \n–Estimate future valuation and fundraising needs  \n–Provide an estimate of employee ownership  \nNote: There is an accompanying excel file that provides a template for \nbuilding an options model similar to that described on the following slide',
   'Options Modeling – A Detailed Example  \nEmployee Stock Options Plan (ESOP) Model\nYear 0 Year 1 Year 2 Year 3 Year 4\nValue Shares Value Shares Shr. Price Value Shares Shr. Price Value Shares Shr. Price\nNA 10,000 $3M 10,000 $300 $5M 10,000 $500 $15M 12,000 $1,250\nOptions Total Equity Grant Total Equity Grant Total Equity Grant Total Equity\nSalary Multiplier Options (%) Amount Options (%) Amount Options (%) Amount Options (%)\nTotal Non-Founder Employee Ownership 6.50% 9.00% 11.64% 10.20%\nFounding Team\nCTO $120,000 1.00x 400 4.00% 400 4.00% $60,000 520 5.20% 520 4.33%\nProduct Manager $80,000 0.80x 250 2.50% 250 2.50% $32,000 314 3.14% 314 2.62%\nYear 1 Hires\nLead Engineer $80,000 0.50x $40,000 133 1.33% 133 1.33% $20,000 149 1.24%\nLead Designer $70,000 0.50x $35,000 117 1.17% 117 1.17% $17,500 131 1.09%\nYear 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires',
   'Year 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires\nMarketing Director $60,000 0.30x $18,000 14 0.12%\nEngineer $50,000 0.25x $12,500 10 0.08%\nSalesperson $40,000 0.10x $4,000 3 0.03%\nOffice Manager $30,000 0.10x $3,000 2 0.02%\nFounding team shares are \nnegotiated as a percentage of the \ncompany; all future employees \nreceive grants determined in dollars  \nRetention grants are awarded every two \nyears after an initial options package  The “true” valuation (an \ninternal estimate) increases \nmarginally as the company \nbuilds value…  …but increases significantly when \nthe company raises a new round, \nwhich also increases the share \ncount and dilutes all equity owners  \nFor all new hires after the \nfounding team, options grants \nare awarded based on the \nemployee’s options multiplier \nand baseline salary',
   'RESOURCES & FURTHER \nREADING  Part V',
   'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity',
   'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting',
   'This work  is licensed  under  the Creative  Commons  Attribution -\nShareAlike  4.0 International  License . To view a copy  of this license,  \nvisit http://creativecommons .org/licenses/by -sa/4.0/ or send  a letter  to \nCreative  Commons,  PO Box 1866 , Mountain  View,  CA 94042 , USA.'],
  [{'chunk': 0,
    'text': 'Startup Employee Stock \nOptions Plans (ESOPs)  \n \nOverview and Best Practices',
    'page number': 'Page1'},
   {'chunk': 0,
    'text': 'Table of Contents  \nPart I: Intro to Options Plans  \n•What is an ESOP?  \n•What is an Option?  \n•Lifecycle of a Startup ESOP  \n•Common Terms in an Options Package  \n•Why Issue Options to Employees ? \n–A Defining Characteristic of Startup Culture  \n–A Necessary Part of the Capital Structure  \n•When to Create an ESOP ? \n•Communicating Options to Employees: % versus $ \n \nPart II: How Much to Grant  \n•Two Approaches  \n•The Top -Down Process  \n–1. How Much Equity to Set Aside in the ESOP ? \n–2. A Typical Distribution Schedule  \n•The Bottom -Up Process  \n–1. Segment Your Human Resources  \n–2. Establish Pay Multipliers for Each Role \n–3. Determine the Dollar Value of the Options Grant  \n–4. Determine the Current Share Price  \n–5. Calculate the Options Grant  \n–An Example: Hiring a CTO  \n•Important Takeaways  Part III : The Fine Print – Terms  \n•Strike Price  \n•Vesting Schedule  \n•The Cliff  \n–Example: Standard Vesting w/ a Cliff \n•Vesting in a Liquidity Event  \n•Exercising Options',
    'page number': 'Page2'},
   {'chunk': 1,
    'text': '•Important Takeaways  Part III : The Fine Print – Terms  \n•Strike Price  \n•Vesting Schedule  \n•The Cliff  \n–Example: Standard Vesting w/ a Cliff \n•Vesting in a Liquidity Event  \n•Exercising Options  \n•Tax Considerations  \n•Legal Advice  \n \nPart IV: ESOPs for the Long Term  \n•Retention Grants  \n•Discretionary Grants  \n•Social Impact Considerations  \n•Options Modeling – Overview  \n•Options Modeling – A Detailed Example  \n \nPart V: Resources & Further Reading',
    'page number': 'Page2'},
   {'chunk': 0,
    'text': 'INTRO TO OPTIONS PLANS  Part I',
    'page number': 'Page3'},
   {'chunk': 0,
    'text': 'What is an ESOP?  \n•An Employee Stock Options Plan (ESOP)  \n \n•An allocation  of shares that will be granted to employees in the future \nin the form of stock options  \n–How much equity should we set aside for employees?  \n \n•A plan for how these options will be distributed:  \n–How many shares will individual employees receive?  \n–What terms will govern these grants?  \n \n•The plan is as important as the allocation!',
    'page number': 'Page4'},
   {'chunk': 0,
    'text': 'What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity',
    'page number': 'Page5'},
   {'chunk': 1,
    'text': 'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time',
    'page number': 'Page5'},
   {'chunk': 0,
    'text': 'Lifecycle of a Startup ESOP  \n•Founders and early investors  create an ESOP by setting aside a \npercentage of shares to be granted to future employees  \n•Management and the Board of Directors  issue these shares to \nemployees as options packages granted for hiring, promotion and \nretention  \n•Employees  receive all of their options upfront, but the company \nmaintains a right to force forfeit that diminishes over time through a \nprocess called “vesting”  \n•Options are exercised  by employees when the company is acquired \nor taken public. The employee pays the “strike price” to acquire the \nshares, but those shares are now marketable at a higher value',
    'page number': 'Page6'},
   {'chunk': 0,
    'text': 'Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants',
    'page number': 'Page7'},
   {'chunk': 0,
    'text': 'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success',
    'page number': 'Page8'},
   {'chunk': 0,
    'text': 'The defining difference between Silicon Valley companies and almost \nevery other industry in the U.S. is the virtually universal practice among \ntech companies of distributing meaningful equity (usually in the form of \nstock options) to ordinary employees .  A Defining Characteristic of Startup Culture  \nSteven Johnson, Technology Writer  \n•Startups are a unique case . Unlike at larger corporations, employee \nownership is an essential element of startup communities and culture  \n–As high -risk/high -reward enterprises, startups use options to align \nemployee compensation with the risk -prone mentality of the \nbusiness  \n–Startups seeking to achieve a “big exit” use options to align all \nemployees to drive toward this desired outcome',
    'page number': 'Page9'},
   {'chunk': 0,
    'text': "A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures",
    'page number': 'Page10'},
   {'chunk': 0,
    'text': 'When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted',
    'page number': 'Page11'},
   {'chunk': 1,
    'text': 'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage',
    'page number': 'Page11'},
   {'chunk': 0,
    'text': 'Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather',
    'page number': 'Page12'},
   {'chunk': 1,
    'text': 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .',
    'page number': 'Page12'},
   {'chunk': 0, 'text': 'HOW MUCH TO GRANT  Part II', 'page number': 'Page13'},
   {'chunk': 0,
    'text': 'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired',
    'page number': 'Page14'},
   {'chunk': 0,
    'text': 'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time',
    'page number': 'Page15'},
   {'chunk': 0,
    'text': '1. How Much Equity to Set Aside in the ESOP?  \n•Founders  are top managers \nwith functional roles that are \nnot redundant  \n•DNA  of the company is in a \nnon-technical industry; no \ntechnology emphasis  \n•Geographic focus  is on a \nregion that is not considered a \nstartup hub  \n•Hiring needs are primarily \nadministrative or back -office \nteam members  <15%  20% 25% Generally non-founder employees  get 15 -20% of the company, with some \ncompanies issuing up to 25% (and a current trend toward bigger ESOPs)  \n•Founders  are top mangers, \nbut hire others to fill key \ntechnical or functional roles  \n•DNA  of the company is in \napplying technology to a non -\ntechnical industry  \n•Geographic focus  is either \nwholly or partially in a startup \nhub \n•Hiring needs are primarily \nsales and marketing team \nmembers  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a',
    'page number': 'Page16'},
   {'chunk': 1,
    'text': 'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP',
    'page number': 'Page16'},
   {'chunk': 0,
    'text': '2. A Typical Distribution Schedule  \nSeniority  Equity Allocation  \nFirst 10 Employees  10% \nNext 20 Employees  5% \nNext 50 Employees  5% \n•Early -stage equity grants are always a negotiation, but generally:  \n–CTO: 1 -5% \n–Key Developer or Engineer: 1 -2% \n–Other Functional Team Member: 0.5 -1.5%  \n–No non -founding member of the senior team should receive over 10%',
    'page number': 'Page17'},
   {'chunk': 0,
    'text': 'The Bottom -Up Process  \n1.Segment your human resources  \n2.Create pay multipliers for each job function  \n3.Determine the dollar value of an options grant  \n4.Determine the current share price  \n5.Calculate options grants',
    'page number': 'Page18'},
   {'chunk': 0,
    'text': '1. Segment Your Human Resources  \nSegment  Roles  Award  \nin % or $?  \nFounders  Co-Founders  % \nFounding Team  Technical & Product  % \nEarly -Stage Hires  Senior Team  (C-Level)  $ \nDirectors / VPs  $ \nFunctional / Technical Team  $ \nSupport Team  $ \n•Think about the early -stage team in three segments  \n–Founders : the founding partners in the business  \n–Founding Team : the people you hire to build your product  \n–Early -Stage Hires : the team you build as you raise capital  \n•As early as possible, begin communicating options grants in terms of \ndollars rather than percentages',
    'page number': 'Page19'},
   {'chunk': 0,
    'text': '2. Establish Pay Multipliers for Each Role  \nSegment  Roles  Award  \nin % or $?  Options Multiplier  \n(Multiple  of Salary)  \nFounders  Co-Founders  % -- \nFounding Team  Technical & Product  % Negotiated  \nEarly -Stage Hires  Senior Team  (C-Level)  $ 0.5x – 1.0x base  salary  \nDirectors / VPs  $ 0.25x – 0.5x base salary  \nFunctional / Technical Team  $ 0.1x – 0.25x base  salary  \nSupport Team  $ 0.0x – 0.1x base salary  \n•After segmenting the early -stage team, establish a standardized \nmultiple of base salary to be granted as an option package  \n–These ranges provide a ballpark grant for each role  \n–Get away from actively negotiating packages with employees  \n–Be transparent about this methodology  \nSource: This range of multiples was provided by Fred Wilson of Union Square Ventures, \nwhich he created in collaboration with a leading HR consulting firm.',
    'page number': 'Page20'},
   {'chunk': 0,
    'text': '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule',
    'page number': 'Page21'},
   {'chunk': 0,
    'text': '4. Determine the Current Share Price  \nLatest “True” Valuation  \nFully -Diluted Share Count  Current Share Price  = What is the company actually worth today?  Or, at what \nvaluation could you raise capital or sell the company? This \nis an internal valuation (not published). Things to consider:  \n-The latest financing round  \n-Any bonafide  acquisition offer  \n-Comparable company valuations / DCF analysis  \nHow many shares are outstanding?  This should include \nall shares issued, as well as any dilutive securities:  \n-Common shares  \n-Preferred shares  \n-Shares authorized for issuance  \n-Options or warrants',
    'page number': 'Page22'},
   {'chunk': 0,
    'text': '5. Calculate the Options Grant  \nOptions Grant  \n(Number of Shares)  = Dollar Value  \nof Options Grant  \nCurrent Share Price',
    'page number': 'Page23'},
   {'chunk': 0,
    'text': '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS',
    'page number': 'Page24'},
   {'chunk': 0,
    'text': 'Important Takeaways  \n•Top-down planning (“the allocation”)  \n–Holistically consider what percentage of the company should \nbelong to employees  \n–Allocate these shares to an ESOP  \n•Bottom -up planning (“the plan”)  \n–Specifically consider how much equity each employee should be \nawarded  \n–Use this framework for individual equity grants  \n•Key Takeaway : Get away from ad hoc equity awards and personal \nnegotiations by standardizing both the amount of equity available to \nemployees and the process by which packages are awarded',
    'page number': 'Page25'},
   {'chunk': 0,
    'text': 'THE FINE PRINT – TERMS  Part III',
    'page number': 'Page26'},
   {'chunk': 0,
    'text': 'Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”',
    'page number': 'Page27'},
   {'chunk': 0,
    'text': 'Vesting Schedule  \n•Vesting protects the company  \n–It stages the accrual of options, mitigating the risk that an employee \nwill depart with an undeserved equity stake  \n–Continually incentivizes employees as they earn their options \npackage over the course of the vesting period  \n•Standard vesting period: 4 years  \n–Alternatively, some companies offer 3 -year vesting, although with a \nshorter vesting period they may offer a smaller options package  The vesting schedule is the timetable over which an employee accrues the \nright to keep the options that have been awarded',
    'page number': 'Page28'},
   {'chunk': 0,
    'text': 'The Cliff  \n•The cliff protects the company  \n•This trial period exists so that the company can avoid giving shares to \nemployees immediately, reserving the opportunity part ways if the \nemployee does not perform well  \n•Standard cliff: 1 year  \n–Alternatively, some companies require a 2 -year cliff. However, in \nstartup communities where employees know what is “standard” this \ncan cause significant discontent  \n–Firing an employee right as their cliff is about to expire is legal, but \nunethical – begin to communicate discontent as early as possible \nduring the trail period  The trial period during which no vesting occurs; in this period vesting \naccrues, but the accrued shares are earned immediately after the cliff',
    'page number': 'Page29'},
   {'chunk': 0,
    'text': '200 shares400 shares600 shares800 shares1,000 shares\n0 months 12 months 24 months 36 months 48 monthsExample: Standard Vesting with a Cliff  \n•New hire  \n•Granted options for 1,000 shares  \n•4-year vesting schedule with 1 -year cliff  \n–At 12 months earns 250 shares  \n–Then earns ~21 shares per month  \nMonths on the Job  Vested Options  \nThe cliff  Monthly vesting',
    'page number': 'Page30'},
   {'chunk': 0,
    'text': 'Vesting in a Liquidity Event  \nThe terms of an options package should specifically address what \nhappens in a liquidity event such as an IPO or acquisition  \n•Vested options generally become exercisable  \n•Unvested options can either:  \n–Roll over and remain unvested  \n–Receive accelerated vesting  \n•These options vest automatically upon change of control – not \ncommon for everyone, but typical for top employees  \n–Receive double -trigger vesting  \n•These options vest automatically upon change of control, but \nonly if the employee is terminated – not common for everyone, \nbut typical for employees who may be extraneous in a merger \n(CFO, General Counsel)',
    'page number': 'Page31'},
   {'chunk': 0,
    'text': 'Exercising Options  \n•Options are “in the money” if the share value is above the strike price  \n•To exercise an option, the employee pays the strike price to the \ncompany and receives shares for each option exercised  \n•If there has been a liquidity event  \n–In the case of an IPO , the employee can now sell these shares in \nthe public markets  \n–In the case of a cash acquisition , an employee’s shares are \nbought out at the cash offer price  \n–In the case of a stock acquisition , an employee’s shares are \nconverted to shares in the new company  Options are usually exercised following a liquidity event, but employees \ncan exercise them at any time after they vest',
    'page number': 'Page32'},
   {'chunk': 0,
    'text': 'Tax Considerations  \nOptions can have material tax consequences for employees; startups \nneed to understand the tax implications and be upfront with new hires  \n•Tax treatment of options will vary by region  \n•Tax treatment will vary by the type option security issued  \n•Several possibilities:  \n–Taxable event upon grant  \n–Taxable event upon leaving company  \n–Taxable event upon exercise  \n–No effective tax event  \n•It is important for employees to fully understand the tax implications of \nthe securities they are receiving',
    'page number': 'Page33'},
   {'chunk': 0,
    'text': 'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language',
    'page number': 'Page34'},
   {'chunk': 0,
    'text': 'ESOPS FOR THE LONG TERM  Part IV',
    'page number': 'Page35'},
   {'chunk': 0,
    'text': 'Retention Grants  \n•Retention grants are used to incentive employees to stay with the \ncompany beyond their 4 -year vesting period: a “top off”  \n–Do not wait until an employee is fully vested to make a retention \ngrant! This creates a natural exit incentive at the end of 4 years  \n–Instead, consider making smaller retention grants every 2 years  \n•A framework for retention grants:  \nGrant Date  Every 2  years, beginning 2 years after initial package  \nOptions Multiplier:  Use ½ of the employee’s standard options multiplier  \nSalary Baseline:  Use current salary  if employee has received a raise   \nValuation & Shares  Use current valuation and share count, if changed  \nRetention  \nGrant  = Current \nSalary  New (Latest) Valuation  \nDiluted Share Count  x 50% of Options \nMultiplier  ÷',
    'page number': 'Page36'},
   {'chunk': 0,
    'text': 'Discretionary Grants  \n•Bonuses Outstanding Performance  \n–These discretionary grants should only be made once per year, and \nonly offered to your top 10 -20% of employees whose \naccomplishments and performance are truly distinguished  \n•Promotions  \n–These discretionary grants should only be made to reward \npromotion. Bring the employee’s total equity up to the amount you \nwould pay to hire them today.',
    'page number': 'Page37'},
   {'chunk': 0,
    'text': 'Social Impact Considerations  \n•Companies focused on social impact goals and/or in developing \nmarkets may have unique ESOP considerations  \n•Consider two possible scenarios:  \n–Financial Inclusion Goals : Employee -friendly hiring practices and \nESOPs can help build local financial inclusion. If this is part of your \ncorporate mandate, consider the added social impact value of \noffering your options program to all levels of employees  \n–Local Ownership Culture : Conversely, in certain regions stock \noptions may have negligible value to employees, either because of \nrisk aversion, lack of liquidity, or lack of understanding. If this is the \ncase, it is not worth extensively offering options to employees who \nwould rather be paid in cash',
    'page number': 'Page38'},
   {'chunk': 0,
    'text': 'Options Modeling – Overview  \n•Creating an ESOP has lasting implications on startup capital structure  \n•While the guidelines provided in this presentation offer a roadmap to \nimplementing an options plan, it can also be helpful to build an options \nmodel to understand the long -term effects of this program  \n•Options models:  \n–Forecast hiring needs over time  \n–Estimate future valuation and fundraising needs  \n–Provide an estimate of employee ownership  \nNote: There is an accompanying excel file that provides a template for \nbuilding an options model similar to that described on the following slide',
    'page number': 'Page39'},
   {'chunk': 0,
    'text': 'Options Modeling – A Detailed Example  \nEmployee Stock Options Plan (ESOP) Model\nYear 0 Year 1 Year 2 Year 3 Year 4\nValue Shares Value Shares Shr. Price Value Shares Shr. Price Value Shares Shr. Price\nNA 10,000 $3M 10,000 $300 $5M 10,000 $500 $15M 12,000 $1,250\nOptions Total Equity Grant Total Equity Grant Total Equity Grant Total Equity\nSalary Multiplier Options (%) Amount Options (%) Amount Options (%) Amount Options (%)\nTotal Non-Founder Employee Ownership 6.50% 9.00% 11.64% 10.20%\nFounding Team\nCTO $120,000 1.00x 400 4.00% 400 4.00% $60,000 520 5.20% 520 4.33%\nProduct Manager $80,000 0.80x 250 2.50% 250 2.50% $32,000 314 3.14% 314 2.62%\nYear 1 Hires\nLead Engineer $80,000 0.50x $40,000 133 1.33% 133 1.33% $20,000 149 1.24%\nLead Designer $70,000 0.50x $35,000 117 1.17% 117 1.17% $17,500 131 1.09%\nYear 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires',
    'page number': 'Page40'},
   {'chunk': 1,
    'text': 'Year 2 Hires\nSales Director $70,000 0.30x $21,000 42 0.42% 42 0.35%\nSalesperson $40,000 0.10x $4,000 8 0.08% 8 0.07%\nEngineer $60,000 0.25x $15,000 30 0.30% 30 0.25%\nYear 3 Hires\nMarketing Director $60,000 0.30x $18,000 14 0.12%\nEngineer $50,000 0.25x $12,500 10 0.08%\nSalesperson $40,000 0.10x $4,000 3 0.03%\nOffice Manager $30,000 0.10x $3,000 2 0.02%\nFounding team shares are \nnegotiated as a percentage of the \ncompany; all future employees \nreceive grants determined in dollars  \nRetention grants are awarded every two \nyears after an initial options package  The “true” valuation (an \ninternal estimate) increases \nmarginally as the company \nbuilds value…  …but increases significantly when \nthe company raises a new round, \nwhich also increases the share \ncount and dilutes all equity owners  \nFor all new hires after the \nfounding team, options grants \nare awarded based on the \nemployee’s options multiplier \nand baseline salary',
    'page number': 'Page40'},
   {'chunk': 0,
    'text': 'RESOURCES & FURTHER \nREADING  Part V',
    'page number': 'Page41'},
   {'chunk': 0,
    'text': 'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity',
    'page number': 'Page42'},
   {'chunk': 0,
    'text': 'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting',
    'page number': 'Page43'},
   {'chunk': 0,
    'text': 'This work  is licensed  under  the Creative  Commons  Attribution -\nShareAlike  4.0 International  License . To view a copy  of this license,  \nvisit http://creativecommons .org/licenses/by -sa/4.0/ or send  a letter  to \nCreative  Commons,  PO Box 1866 , Mountain  View,  CA 94042 , USA.',
    'page number': 'Page44'}],
  ['what is a neccessary Part of the Capital Structure according to Fred Wilson?',
   'When to Communicate Options to Employees: % versus $',
   'When to Create an ESOP?',
   'What are the Common Terms in an Options Package?']),
 HuggingFaceEmbeddings(client=SentenceTransformer(
   (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
   (2): Normalize()
 ), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}))

SyntaxError: invalid syntax (<ipython-input-24-75b9c453b58f>, line 31)

In [28]:
%%time
out = postprocess_function(response, content_type=None)
#
predictions,database_df,query_df = out

Content : ESOP GUIDE - INDIA
7
COPYRIGHT, CAKE EQUITY 2022Attracting Talent.
It’s no secret that cash can be tight for startups, which makes it hard to pay top 
dollar for talent. Using an ESOP, start-ups can offer tasty equity incentives to 
top-up slightly lower remuneration packages. The same applies for attracting 
experienced advisors and developers at an early stage. Knock their socks off with 
your idea, pay them in Options.   
Retaining Talent.
It’s hard to attract top talent and even harder to keep it. Options are usually 
subject to Vesting, which encourages sticking with a company until the Options 
Vest. That’s why it’s often called “sweat equity” – team members need to put in 
the blood, sweat and tears in order to eat their cake. 
 
Opportunity for significant employee financial gain.
By taking a slice of the ESOP cake, an employee can benefit from the increase 
in value of the stocks over the company lifetime. It offers hungry employees the 
chance to generate real wealt

In [26]:
#response
{'Query': ['what is a neccessary Part of the Capital Structure according to Fred Wilson?', 'When to Communicate Options to Employees: % versus $', 'When to Create an ESOP?', 'What are the Common Terms in an Options Package?'], 'Answer': ['According to Fred Wilson, a necessary part of the capital structure is employee equity or ESOPs (Employee Stock Ownership Plans). He states, "I can\'t think of a term sheet that we have issued that didn\'t have a specific provision for employee equity." This suggests that offering employee equity is a prerequisite for venture capitalists and is commonly included in deals with startups.', 'The context does not provide information about when to communicate options to employees as a percentage versus a dollar value.', 'ESOPs should be created during the seed stage or early-VC stage of a startup.', 'The common terms in an options package are:\n1. Number of Shares: The total number of options granted to an employee, and the maximum number of shares that employee has access to.\n2. Strike Price: The price the employee must pay to purchase each share if and when the employee chooses to exercise the option.\n3. Vesting Schedule: The timeline over which the options become wholly owned and exercisable by the employee, no longer subject to repurchase by the company.\n4. Cliff Period: The trial period during which no vesting occurs; in this period vesting accrues, but the total effect of this vesting is realized immediately after the cliff.\n5. Expiration Date: The last date on which the options may be exercised and converted into common shares by the employee.\n\nQuotes supporting the answer:\n- "Number of Shares: The total number of options granted to an employee, and therefore the maximum number of shares that employee has access to."\n- "Strike Price: The price the employee must pay to purchase each share if and when the employee chooses to exercise the option."\n- "Vesting Schedule: The timeline over which the options become wholly owned and exercisable by the employee (no longer subject to repurchase by the company)."\n- "Cliff Period: The trial period during which no vesting occurs; in this period vesting accrues, but the total effect of this vesting is realized immediately after the cliff."\n- "Expiration Date: The last date on which the options may be exercised and converted into common shares by the employee."'], 'Referrences': [["A Necessary Part of the Capital Structure  \n•Venture capitalists require ESOPs . For many VCs, establishing a \nstock option pool is a prerequisite to closing a deal  \n–In an industry where options are ubiquitous, startups are compelled \nto offer options packages to compete for top talent with other \nventure -backed companies  \n–When operating budgets are tight, competitive compensation \npackages may not be possible; options can be used to incentivize \nemployees instead of cash  \nI can't think of a term sheet that we have issued that didn't have a \nspecific provision for employee equity.  \nFred Wilson, Union Square Ventures", 'The Top -Down Process  \n1.Determine how much equity to set aside for non -founder employees  \n2.Create a schedule of how this equity will be distributed over time', 'Resources & Further Reading  \n•Fred Wilson / Union Square Ventures Series on Employee Equity  \n–What are options?  \n–Structures and allocations  \n–How many options to issue?  \n–Vesting, cliffs, and change of control  \n–Trends in increasing issuance size  \n–Dilution  \n–Video: Class on Employee Equity', 'Resources & Further Reading  \n•Andy Rachleff  / Wealthfront  materials  \n–The Wealthfront  startup equity plan  \n–The Wealthfront  equity tool for benchmarking startup options \npackages  \n•Sam Altman and Paul Graham / Y -Combinator  \n–Why employees don’t get enough options  \n–The “Equity Equation”  \n•Startup Lawyer  \n–Vesting', 'Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively pursuing \ngrowth and hiring; likely to have \nexhausted most of the ESOP  Most of the ESOP is gone, but shares \nremaining are more valuable; use them \nto allow  new hires to share in the upside  Make employee equity allocations and set up an ESOP sometime \nbetween the pre -seed and early -VC stage'], ['Communicating Options to Employees: % versus $  \nOptions packages can be communicated either as (1) a percentage \nof ownership in the company, or (2) a dollar value based on the \ncurrent valuation; we encourage the latter when possible  \n% $ \n“We are granting you options equivalent to \n0.5% of the company’s equity”  \nConsiderations:  \n•At an early stage, the only way  to \ncommunicate options grants given no true \nvaluation of the company  \n•Hard for employees to grasp  what they \nare really getting (“0.5% seems too little”)  \n•Invites a negotiation  about what percent \nof the company an employee really \ndeserves  “We are granting you options equivalent to \n$200,000 of company stock”  \nConsiderations:  \n•Once valuation is established, highly \neffective  way to communicate the true \nvalue of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather', 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', 'Why Issue Options to Employees?  \n•Attract Talent:  options can be used to attract top recruits, particularly \nengineers, product managers and other technical team members  \n•Retain Employees:  options vest over several years, creating strong \nincentives for employees to remain employees  \n•Align Incentives:  by making employees equity owners, options align \nincentives with the long -term goals of the company  \n•Reward Value Creation:  options reward tangible contributions that \nincrease corporate valuation by giving employees a slice of that value  \n•Encourage Long -term Thinking : options typically pay off only in a \nliquidity event or exit, and thus push employees to build the company \nfor long -term success', '0.8 An Example: Hiring a CTO  \nEmployee Information  \n•CTO (Senior Team)  \n•$100,000 salary  \n•0.8x options multiplier  \nCompany Information  \n•Last valuation round: $5M  \n•Amount raised: $2M  \n•Current valuation: $7M  \n•Diluted share count: 50,000  Dollar Value of \nOptions Grant  = $100,000  x $80,000  = \nCurrent  \nShare Price  = $7M \n$140  = \n50,000  \nOptions Grant  = $80,000  \n571 shares  = \n$140  \nNote that this employee received 1.1% of the diluted shares outstanding; \nhowever, the grant is communicated as an $80,000 grant of 571 options, \nrather than as a percentage ownership stake in the company  CALCULATION STEPS', '3. Determine the Dollar Value of the Options Grant  \nEmployee  \nBase Salary  = x Options  \nMultiplier  Dollar Value  \nof Options Grant  Base Salary as negotiated with \nthe employee at the time of hire \nor promotion  \nOptions Multiplier is decided \nbased on the employee’s role in \nthe HR segmentation schedule'], ['When to Create an ESOP?  \nStage  Considerations  Takeaway  \nPre-seed  Founders focused on traction  (often too \nbusy for an ESOP).  Key employees are \ngiven equity/options on an ad hoc basis  ESOP  not necessary, but it can be \nhelpful to sanity check how much equity \nyou are giving away to early hires  \nSeed  First outside financing round. Investors \neither angel or institutional; institutional \ninvestors will require an ESOP  Seed rounds can be closed without an \nESOP; the benefit to doing so is that \nseed investors then share in the dilution  \nEarly -VC The first true VC round. Investors will \nrequire an ESOP in place. New hires \nwill be seek large equity grants.  ESOP must be created  (to appease \ninvestors and to serve as a guideline for \nthe size of new -hire options grants)  \nLate-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted', 'Two Approaches  \nIn reality, creating an ESOP will require a combination of \ntop-down and bottom -up planning  Top Down  Bottom Up  \nDecide the total amount of \nequity to be granted; allocate \nthese shares to employees \nover time  Decide the appropriate size of \nindividual equity grants by \nposition; issue these shares as \nemployees are hired', 'members  •Founders  are not top \nmanagers in the business, or \nplan to step aside  \n•DNA  of the company is in \ntechnology, engineering, or \ndata science  \n•Geographic focus  is in a \nleading startup hub such as \nSF, NYC or Boston  \n•Hiring needs are primarily \nengineering, development, or \ntechnical team members  \nThere are no hard and fast rules, but by looking your company’s founders, its DNA, \nits geographic focus, and its hiring needs you can begin to benchmark how much \nequity to set aside for the ESOP', 'Legal Advice  \nWhen structuring an ESOP, engage an experienced \nstartup lawyer with regional expertise to standardize the \nterms and language', 'ESOPS FOR THE LONG TERM  Part IV'], ['Common Terms in an Options Package  \nNumber of Shares  The total number of  options granted to an \nemployee, and therefore the maximum number of \nshares that employee has access to   \nStrike Price  The price the  employee must pay to purchase \neach share if and when the employee chooses to \nexercise the option  \nVesting  Schedule  The timeline over which the options become \nwholly owned and exercisable  by the employee \n(no longer subject to repurchase by the company)  \nCliff Period  The trial  period during which no vesting occurs; in \nthis period vesting accrues, but the total effect of \nthis vesting is realized immediately after the cliff  \nExpiration Date  The last date on which the options  may be \nexercised and converted into common shares by \nthe employee  \nWe will discuss the mechanics in further detail, but these basic \nterms are helpful to understanding options grants', 'Appropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity \nof a company’s stock at a set price for a certain period of time', 'What is an Option?  \n•Why do options have intrinsic value?  \n–A effective form of equity ownership  \n–A locked -in price for shares  \n•How do startups use options?  \n–To bring in founding team members \nwho are not co -founders  \n–To recruit, compensate and retain \nearly employees  \n–To allow later employees to share in \nthe company’s long -term upside  Terminology:   \nThis presentation uses “options” \ngenerally to refer to several types of \nsecurities that are often issued to startup \nemployees to provide for effective equity \nownership, including:  \n-Stock options (the right to buy \ncommon stock a set strike price)  \n-Restricted stock (common stock \nissued early on to top employees)  \n-Restricted stock units (a promise to \nissue common stock in the future)  \nAppropriate use of these securities will \nvary based on local regulatory and tax \nconsiderations.  An option is a right (but not an obligation) to purchase a quantity', 'value of an options grant  \n•Easy for employees to grasp  what they \nare really getting (“$200,000 is a lot”)  \n•Grounds negotiations  in a discussion \nabout concrete dollar amounts, rather \nthan a percent of the company  \nNote: Although we recommend communicating options packages as a dollar value, we do not suggest \nrefusing employees information about their effective percentage ownership ; all options holders are \nentitled to this information, and to refuse would be unethical .', 'Strike Price  \n•Options are “struck” at a specific strike price when issued; the holder \nmust pay this amount in order to exercise the option  \n–The expectation is that the shares will have significantly increased \nin value, and the holder will profit from the spread  \n–Regulations for how strike prices can be set will vary by region  \n•However, it is generally in the best interest of both company and \nemployee if the strike price is set as low as possible  \n–This is because options are typically exercised only after an \nacquisition or IPO, so while a higher strike price costs the \nemployee more, none of the current shareholders benefit  The strike price is the amount that the employee must pay to turn one \noption into one share of stock. Also known as the “exercise price”']], 'Page Numbers': [['Page10', 'Page15', 'Page42', 'Page43', 'Page11'], ['Page12', 'Page12', 'Page8', 'Page24', 'Page21'], ['Page11', 'Page14', 'Page16', 'Page34', 'Page35'], ['Page7', 'Page5', 'Page5', 'Page12', 'Page27']], 'Scores': [[0.4914062023162842, 0.4569064974784851, 0.4016406834125519, 0.39973437786102295, 0.39906424283981323], [0.7542568445205688, 0.5692958235740662, 0.4829898476600647, 0.4591796398162842, 0.4469922184944153], [0.73818039894104, 0.7000071406364441, 0.6700037121772766, 0.6582071781158447, 0.6364336013793945], [0.664095401763916, 0.5762392282485962, 0.5053941011428833, 0.49215400218963623, 0.48697030544281006]]}


{'Query': ['what is a neccessary Part of the Capital Structure according to Fred Wilson?',
  'When to Communicate Options to Employees: % versus $',
  'When to Create an ESOP?',
  'What are the Common Terms in an Options Package?'],
 'Answer': ['According to Fred Wilson, a necessary part of the capital structure is employee equity or ESOPs (Employee Stock Ownership Plans). He states, "I can\'t think of a term sheet that we have issued that didn\'t have a specific provision for employee equity." This suggests that offering employee equity is a prerequisite for venture capitalists and is commonly included in deals with startups.',
  'The context does not provide information about when to communicate options to employees as a percentage versus a dollar value.',
  'ESOPs should be created during the seed stage or early-VC stage of a startup.',
  'The common terms in an options package are:\n1. Number of Shares: The total number of options granted to an employee, and the maximum number of s

In [29]:
import pandas as pd

pd.set_option("display.max_colwidth", 50)
database_df.head()
query_df.head()
cen =[]
c_text_vector =[]
for items in  database_df["text_vector"].values:
  cen.append(np.mean(items))
  c_text_vector.append(items - np.mean(items))
#
database_df["centered_text_vector"]  = c_text_vector
database_df.head()



,text,text_vector,centered_text_vector
0,"ESOP GUIDE - INDIA\n1\nCOPYRIGHT, CAKE EQUITY ...","[0.0006247127894312143, 0.012729390524327755, ...","[0.0006398774957349085, 0.012744555230631449, ..."
1,"ESOP GUIDE - INDIA\n2\nCOPYRIGHT, CAKE EQUITY ...","[0.0006247127894312143, 0.012729390524327755, ...","[0.0006398774957349085, 0.012744555230631449, ..."
2,get your head around standard terms for startu...,"[-0.0060834153555333614, 0.030363131314516068,...","[-0.00580626511387361, 0.03064028155617582, -0..."
3,"ESOP GUIDE - INDIA\n3\nCOPYRIGHT, CAKE EQUITY ...","[0.0006247127894312143, 0.012729390524327755, ...","[0.0006398774957349085, 0.012744555230631449, ..."
4,"ESOP GUIDE - INDIA\n4\nCOPYRIGHT, CAKE EQUITY ...","[0.0006247127894312143, 0.012729390524327755, ...","[0.0006398774957349085, 0.012744555230631449, ..."


In [30]:
qcen =[]
qc_text_vector =[]
for items in  query_df["text_vector"].values:
  qcen.append(np.mean(items))
  qc_text_vector.append(items - np.mean(items))
#
query_df["centered_text_vector"] = qc_text_vector
query_df.head()

,text,text_vector,response,context_text_0,context_text_1,context_similarity_0,context_similarity_1,user_feedback,centered_text_vector
0,what is a neccessary Part of the Capital Struc...,"[0.0180486049503088, 0.004823063034564257, -0....",I don't know the answer to this question as th...,"ESOP GUIDE - INDIA\n7\nCOPYRIGHT, CAKE EQUITY ...","ESOP GUIDE - INDIA\n5\nCOPYRIGHT, CAKE EQUITY ...",0.433168,0.409293,1.0,"[0.01806579956634761, 0.004840257650603068, -0..."
1,When to Communicate Options to Employees: % ve...,"[0.021837031468749046, 0.01641995459794998, -0...",The provided context does not contain any info...,"when it Exercises the Option, and it is often ...",most recent valuation of the company. Role v T...,0.472564,0.417693,1.0,"[0.02167420279867079, 0.016257125927871725, -0..."
2,When to Create an ESOP?,"[0.007623074576258659, -0.021725615486502647, ...","Based on the provided context, the answer to t...","ESOP GUIDE - INDIA\n23\nCOPYRIGHT, CAKE EQUITY...","By taking a slice of the ESOP cake, an employe...",0.630602,0.619418,1.0,"[0.007500564434783752, -0.021848125627977555, ..."
3,What are the Common Terms in an Options Package?,"[-0.023321883752942085, -0.1582910567522049, -...",The common terms in an options package include...,(listing) or acquisition by a third party. \nG...,Agreement or an Option Agreement. \nOption mea...,0.601347,0.574396,1.0,"[-0.023122343940304362, -0.15809151693956716, ..."


In [33]:
import os
import textwrap
from datetime import timedelta
from typing import Dict, List, Optional, Tuple
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from tqdm import tqdm

pd.set_option("display.max_colwidth", None)
#
EVALUATION_SYSTEM_MESSAGE = "You will be given a query and a reference text. You must determine whether the reference text contains an answer to the input query. Your response must be binary (0 or 1) and should not contain any text or characters aside from 0 or 1. 0 means that the reference text does not contain an answer to the query. 1 means the reference text contains an answer to the query."
QUERY_CONTEXT_PROMPT_TEMPLATE = """# Query: {query}

# Reference: {reference}

# Binary: """

openai.api_key = os.environ["OPENAI_API_KEY"]
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def evaluate_query_and_retrieved_context(query: str, context: str, model_name: str) -> str:
    prompt = QUERY_CONTEXT_PROMPT_TEMPLATE.format(
        query=query,
        reference=context,
    )
    response = openai.chat.completions.create(
        messages=[
            {"role": "system", "content": EVALUATION_SYSTEM_MESSAGE},
            {"role": "user", "content": prompt},
        ],
        model=model_name,
    )
    return response.choices[0].message.content


def evaluate_retrievals(
    retrievals_data: Dict[str, str],
    model_name: str,
) -> List[str]:
    responses = []
    for query, retrieved_context in tqdm(retrievals_data.items()):
        response = evaluate_query_and_retrieved_context(query, retrieved_context, model_name)
        responses.append(response)
    return responses


def process_binary_responses(
    binary_responses: List[str], binary_to_string_map: Dict[int, str]
) -> List[str]:
    """
    Parse binary responses and convert to the desired format
    converts them to the desired format. The binary_to_string_map parameter
    should be a dictionary mapping binary values (0 or 1) to the desired
    string values (e.g. "irrelevant" or "relevant").
    """
    processed_responses = []
    for binary_response in binary_responses:
        try:
            binary_value = int(binary_response.strip())
            processed_response = binary_to_string_map[binary_value]
        except (ValueError, KeyError):
            processed_response = None
        processed_responses.append(processed_response)
    return processed_responses

#Invoke the functions defined for Evaluation
sample_query_df = query_df.head(10).copy()
evaluation_model_name = "gpt-3.5-turbo"  # use GPT-4 if you have access
for context_index in range(2):
    retrievals_data = {
        row["text"]: row[f"context_text_{context_index}"] for _, row in sample_query_df.iterrows()
    }
    raw_responses = evaluate_retrievals(retrievals_data, evaluation_model_name)
    processed_responses = process_binary_responses(raw_responses, {0: "irrelevant", 1: "relevant"})
    sample_query_df[f"openai_relevance_{context_index}"] = processed_responses
sample_query_df[
    ["text", "context_text_0", "openai_relevance_0", "context_text_1", "openai_relevance_1"]
].head(10)

100%|██████████| 4/4 [00:02<00:00,  1.62it/s]


,text,context_text_0,openai_relevance_0,context_text_1,openai_relevance_1
0,what is a neccessary Part of the Capital Structure according to Fred Wilson?,"ESOP GUIDE - INDIA\n7\nCOPYRIGHT, CAKE EQUITY 2022Attracting Talent.\nIt’s no secret that cash can be tight for startups, which makes it hard to pay top \ndollar for talent. Using an ESOP, start-ups can offer tasty equity incentives to \ntop-up slightly lower remuneration packages. The same applies for attracting \nexperienced advisors and developers at an early stage. Knock their socks off with \nyour idea, pay them in Options. \nRetaining Talent.\nIt’s hard to attract top talent and even harder to keep it. Options are usually \nsubject to Vesting, which encourages sticking with a company until the Options \nVest. That’s why it’s often called “sweat equity” – team members need to put in \nthe blood, sweat and tears in order to eat their cake. \n \nOpportunity for significant employee financial gain.\nBy taking a slice of the ESOP cake, an employee can benefit from the increase \nin value of the stocks over the company lifetime. It offers hungry employees the \nchance to generate real wealth.",irrelevant,"ESOP GUIDE - INDIA\n5\nCOPYRIGHT, CAKE EQUITY 2022ESOPs are the most popular way for startups \nand small businesses to slice up their equity \ncakes because it creates less paperwork for \nthe business and better tax outcomes for the \nperson receiving the Options. \nInvestors usually require an ESOP is \nimplemented before they inject cash into a startup because they know how effective it \nis for attracting and retaining talent. And, of \ncourse, they don’t want to see their equity \ndiluted down the track. \nA Piece of Cake.\nEquity can be faster, simpler, and easier -- dare \nwe say it, a piece of cake, with our end-to-end \nplatform that allows you to plan, create, approve \nand manage your ESOP.\nOn average, our solution takes 85% less time \nand costs 90% less money than the average DIY \nequivalent, and you can get set up in minutes.\nCake works closely with expert start-up lawyers \nand accountants around the world. If you want \nsome extra advice, we can put you in touch with",irrelevant
1,When to Communicate Options to Employees: % versus $,"when it Exercises the Option, and it is often a \nnominal value. \nHow many Options \nshould I allocate to \nmy team members?No two companies are the same, so it is \nimportant to specifically consider your own \nstaff, and your plans for the next few years. \nAbove all, every company should keep team \nincentivisation top-of-mind.",irrelevant,"most recent valuation of the company. Role v Time\nWhile Option allocations will sometimes vary \nbetween roles, it’s more common for them to \nbe based on time spent in the company. For \nexample, the early employees will have higher \nallocations, and the later employee allocations \nwill be smaller, despite them being in similar \nroles. Although this isn’t to say late joiners can’t \nget higher allocations – it all comes down to \nthe specific values of the team member and \ncompany goals. \nCommon Percentages\nIt’s quite rare for an employee or contractor to \nreceive more than 1% fully diluted ownership in \na company, except where that team ember is a \nsenior hire, or the Options are in substitution for \na significant salary package.\nPlease keep in mind that where the recipient \nis an employee, they must be paid at least the \nminimum salary under the employment laws \nrelevant to your jurisdiction and this salary \ncannot be substituted with Options in most \njurisdictions.",irrelevant
2,When to Create an ESOP?,"ESOP GUIDE - INDIA\n23\nCOPYRIGHT, CAKE EQUITY 2022We make equity easier to use.\nOn average, our solution takes 85% less time and costs 90% less money \nthan the average DIY equivalent. \nOur ESOP feature allows you to plan, approve, implement, issue and \nmanage your ESOP in minutes. Our recipe? The perfect blend of \nautomation and professional advice from expert